In [1]:
import os
from groq import Groq
from IPython.display import display, Markdown
import wikipediaapi
import requests
from bs4 import BeautifulSoup
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
# Function to fetch Wikipedia page content in HTML format by URL
def fetch_wikipedia_page_by_url(url):
    headers = {
        'User-Agent': 'prompt engineering'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Extract the text content from the HTML, ignoring tables, images, etc.
        paragraphs = soup.find_all('p')
        text_content = '\n\n'.join([para.get_text() for para in paragraphs])
        
        return text_content
    else:
        return None

In [3]:
# URL of the Wikipedia page you want to fetch
url = "https://en.wikipedia.org/wiki/Samsung"

# Fetch the HTML content of the page
samsung_html_content = fetch_wikipedia_page_by_url(url)

# Check if the content was fetched successfully and print it
if samsung_html_content:
    print("Content fetched!")
else:
    print(f"Page at '{url}' not found or could not be fetched.")

Content fetched!


In [88]:
# Create an instance of RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000,
    chunk_overlap=2000
)

# Split the text into chunks and store the output
samsung_split = splitter.create_documents([samsung_html_content])

# Write the chunks to a .txt file
with open('samsung_split.txt', 'w', encoding='utf-8') as file:
    for doc in samsung_split:
        # Remove any extra newlines at the end of the chunk
        cleaned_content = doc.page_content.rstrip('\n')
        file.write(cleaned_content + "\n" + "=" * 100 + "\n")

In [5]:
parser = StrOutputParser()

In [6]:
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm = ChatGroq(model = 'llama3-8b-8192')

In [268]:
summarize_chunk_prompt = """Your task is to extract important information from
the provided chunk of context and include the specific sentence from which each
piece of information was derived. Do not update or modify the source sentences.
Let's follow these steps:
Step 1: Identify Key Information: Carefully read the chunk and extract the 
following details: "Name", "Founders", "Founded", "Headquarters",
"Industry", "Products", "Services", "Revenue", "Timeline".
Step 2: Extract Information: Extract the relevant information in the 
chunk and pair it with its source sentence. The source sentence should not be
modified and should be copied word-for-word from the chunk of information.
Step 3: Formatting :Present the extracted information, except "Timeline", in 
the following format:
<Information_Type>: <information>
    Source sentence: <sentence_where_you_got_the_information_from>
Step 4: "Timeline" should be formatted in the following format:
Timeline:
    <event_year>: <event>
            <source>: <sentence_where_you_got_event_information_from>

IMPORTANT: Cite Information: For each key detail, ensure to include the exact 
source sentence from which the information was derived. Only include details if
there is an available source sentence.

IMPORTANT: ONLY INCLUDE INFORMATION THAT HAS A CORRESPONDING SOURCE SENTENCE.
Chunk:
###
{chunk}
###
"""

In [269]:
def process_chunk_with_llm(chunk, llm, prompt):
    prompt_template = ChatPromptTemplate.from_messages(
        [("user", prompt)]
    )
    llmchain = prompt_template | llm | parser
    return llmchain.invoke(chunk)

In [270]:
samsung_responses = []
for i, chunk in enumerate(samsung_split):
    samsung_responses.append(process_chunk_with_llm(chunk, llm))

In [272]:
Markdown(samsung_responses[-1])

Here is the extracted information:

Name: None (not mentioned in the provided chunk)

Founders: None (not mentioned in the provided chunk)

Founded: None (not mentioned in the provided chunk)

Headquarters: None (not mentioned in the provided chunk)

Industry: None (not mentioned in the provided chunk)

Products: None (not mentioned in the provided chunk)

Services: None (not mentioned in the provided chunk)

Revenue: None (not mentioned in the provided chunk)

Timeline:
    1998: Samsung companies were part of a price cartel for DRAMs
            Source: "On 19 October 2011, Samsung companies were fined €145,727,000 for being part of a price cartel of ten companies for DRAMs which lasted from 1 July 1998 to 15 June 2002. The companies received, like most of the other members of the cartel, a 10% reduction for acknowledging the facts to investigators."
    2002: Price fix investigated and ended
            Source: "A recession started to occur that year, and the price fix ended; however, in 2014, the Canadian government reopened the case and investigated silently."
    2011: Samsung fined €145,727,000
            Source: "On 19 October 2011, Samsung companies were fined €145,727,000 for being part of a price cartel of ten companies for DRAMs which lasted from 1 July 1998 to 15 June 2002."
    2014: Canadian government reopened the price fix case
            Source: "A recession started to occur that year, and the price fix ended; however, in 2014, the Canadian government reopened the case and investigated silently."
    2016-2018: Samsung advertised its Galaxy S7, S7 Edge, A5, A7, S8, S8 Plus and Note 8 devices as able to survive short immersion in water
            Source: "However, after many user complaints about the devices having issues after water submersion, such as charger port corrosion. The ACC have officially labelled the fact these devices have 'water resistance' listed as a feature misleading and proceeded with the fine."
    2016: South Korean political scandal involving Samsung
            Source: "The investigative team of special prosecutors looking into the 2016 South Korean political scandal announced that the Blue House received money from South Korea's four largest chaebols (Samsung, Hyundai Motor Group, SK Group and LG Group) to fund pro-government demonstrations by conservative and far-right organizations such as the Korean Parent Federation (KPF) and the Moms Brigade."
    2022: Australia's competition and consumer commission fined Samsung AU$14 million
            Source: "In Australia during 2022, Australia's competition and consumer commission fined Samsung AU$14 million."

In [284]:
refine_prompt = """You will be given a context and a new chunk of new 
information. Your job is to add the new important details to the context from
the new chunk.
For each piece of information in the chunk there will be a source sentence 
relating to that piece of information.
IMPORTANT: THE SOURCE SENTENCES SHOULD ALWAYS BE COPIED WORD FOR WORD.
Let's think step-by-step.
Step1: Read and Understand: Carefully read the provided chunk and its source
sentence to understand all the important points.
Step 2: Analyze New Information: Analyze the new chunk of information and
identify important informations and their respective source sentences.
Step 3: Extract Information:Note down these important details if available:
"Name", "Founders", "Founded", "Headquarters", "Industry", "Products",
"Services", "Revenue", "Timeline".
Step 4: Merge Information: Add the new information with the existing context, 
ensuring that no important detail is omitted. Be as detailed as possible while 
maintaining logical flow and consistency.
Step 5: Maintain Natural Readability: Ensure the final summary reads naturally,
maintains logical flow and consistency, and avoids repetition.
Step 6: Formatting: Present the final context with each piece of information 
followed by its source sentence, formatted as shown below. Source sentence
should always be included in the output and not referenced:
<Information>: <detail>
    Source sentence: <respective_source_sentence_of_the_information>
IMPORTANT: DO NOT FORGET THE IMPORTANT INFORMATION THAT IS ALREADY PRESENT IN
THE CONTEXT. ONLY ADD NEW RELEVANT INFORMATION FROM THE CHUNK TO THE OLD 
CONTEXT
###
Context: {context}
###
###
Chunk: {chunk}
###"""

In [285]:
def refine(chunk, context):
    """
    Refines and merges important details from a given context and a new chunk 
    of information to create a comprehensive and coherent summary.
    The function processes the context and the new chunk of information using a
    language model (LLM) and returns a detailed and accurate summary that 
    includes all critical points.

    Parameters:
    chunk (str): New chunk of information to be integrated with the context.
    context (str): Existing context that provides background or previous
    information.

    Returns:
    str: A refined summary that integrates information from both the context 
    and the new chunk.
    """
    prompt_template = ChatPromptTemplate.from_messages(
        [("user", refine_prompt)]
    )
    llmchain = prompt_template | llm | parser
    return llmchain.invoke({"context": context, "chunk": chunk})

In [275]:
context_samsung = ""
for response in samsung_responses:
    context_samsung = refine(response, context_samsung)

In [276]:
len(context_samsung)

17410

In [277]:
Markdown(context_samsung)

I will merge the new information with the existing context. Here is the updated context:

**Name**: Samsung

Source sentence: None (not mentioned in the provided chunk)

**Founders**: Lee Byung-chul, Lee Kun-hee

Source sentences:
    - Hotel Shilla (also known as "The Shilla") opened in March 1979, following the intention of the late Lee Byung-chul, founder of the Samsung Group.[102]
    - None (not mentioned in the provided chunk)

**Founded**: Various dates

Source sentences:
    - Samsung Fire & Marine Insurance was founded in January 1952 as Korea Anbo Fire and Marine Insurance and was renamed Samsung Fire & Marine Insurance in December 1993.[81]
    - Samsung Engineering is a multinational construction company headquartered in Seoul, founded in January 1969.[78]
    - Samsung Heavy Industries is a shipbuilding and engineering company headquartered in Seoul, founded in August 1974.[83]
    - Samsung SDS is a multinational IT Service company, founded in March 1985.[100]
    - Samsung Advanced Institute of Technology (SAIT), established in 1987, is headquartered in Suwon and operates research labs around the world.[100]
    - Samsung Medical Center was founded on 9 November 1994, under the philosophy of "contributing to improving the nation's health through the best medical service, advanced medical research and development of outstanding medical personnel".[103]
    - None (not mentioned in the provided chunk)

**Headquarters**: Suwon, South Korea (Samsung Electronics)

Source sentence: Samsung Electronics is a multinational electronics and information technology company headquartered in Suwon and the flagship company of the Samsung Group.[69]

**Industry**: Electronics and information technology, construction, shipbuilding and engineering, general insurance, life insurance, biopharmaceuticals, Environment & Asset, Food Culture, Resort, Medical, Chemical, Aerospace, Defense, Technology, etc.

Source sentences:
    - Samsung Electronics is a multinational electronics and information technology company headquartered in Suwon and the flagship company of the Samsung Group.[69]
    - Samsung Engineering is a multinational construction company headquartered in Seoul.[78]
    - Samsung Heavy Industries is a shipbuilding and engineering company headquartered in Seoul.[83]
    - Samsung Fire & Marine Insurance is a multinational general insurance company headquartered in Seoul.[80]
    - Samsung Life Insurance is a multinational life insurance company headquartered in Seoul.[86]
    - Samsung Biologics is a biopharmaceutical division of Samsung, founded in 2011.[74]
    - page_content='Samsung C&T Corporation covers the three main sectors of Environment & Asset, Food Culture and Resort.

**Products**: Air conditioners, computers, digital television sets, active-matrix organic light-emitting diodes (AMOLEDs), mobile phones, display monitors, computer printers, refrigerators, semiconductors and telecommunications networking equipment, Chemical products, medical equipment, microprocessors, plasma displays, steel, etc.

Source sentences:
    - Its products include air conditioners, computers, digital television sets, active-matrix organic light-emitting diodes (AMOLEDs), mobile phones, display monitors, computer printers, refrigerators, semiconductors and telecommunications networking equipment.[70]
    - None (not mentioned in the provided chunk)

**Services**: IT services, insurance services, biopharmaceutical services, construction services, shipbuilding and engineering services, medical services, research and development, manufacturing, sales, etc.

Source sentences:
    - Samsung SDS is a multinational IT Service company, founded in March 1985.[100]
    - Samsung Fire & Marine Insurance offers services including accident insurance, automobile insurance, casualty insurance, fire insurance, liability insurance, marine insurance, personal pensions and loans.[82]
    - Samsung Biologics has contract development and manufacturing (CDMO) services including drug substance and product manufacturing and bioanalytical testing services.[74]
    - Samsung Engineering is the construction of oil refining plants; upstream oil and gas facilities; petrochemical plants and gas plants; steel making plants; power plants; water treatment facilities; and other infrastructure.[78]
    - Samsung Heavy Industries is a shipbuilding and engineering company headquartered in Seoul, founded in August 1974.[83]
    - In 2010, the Samsung Medical Center and pharmaceutical multinational Pfizer agreed to collaborate on research to identify the genomic mechanisms responsible for clinical outcomes in hepatocellular carcinoma.[106][107]

**Revenue**:

* $14 billion (U.S.) on advertising and marketing in 2013
* Over 306 billion USD in export revenue to Vietnam from 2018 to 2022
* 65 billion USD in export revenue to Vietnam in 2022

Source sentences:
    - "Samsung Electronics spent an estimated $14 billion (U.S.) on advertising and marketing in 2013."
    - "In the period from 2018 to 2022, Samsung contributed over 306 billion USD in export revenue to Vietnam."
    - "In 2022 alone, despite the impact of the COVID-19 pandemic, the figure reached 65 billion USD"

**Timeline**:

1980s-1990s: Samsung manufactured a range of aircraft

Source sentence: The company was founded in 1999 as Korea Aerospace Industries (KAI), the result of a merger between then three domestic major aerospace divisions of Samsung Aerospace, Daewoo Heavy Industries and Hyundai Space and Aircraft Company. However, Samsung still manufactures aircraft engines and gas turbines.[35]

1987: Samsung Group was separated into five business groups

Source sentence: After Lee, the founder's death in 1987, Samsung Group was separated into five business groups – Samsung Group, Shinsegae Group, CJ Group, Hansol Group and the JoongAng Group.[26]

1992: Samsung became the world's largest producer of memory chips

Source sentence: Samsung became the world's largest producer of memory chips in 1992 and is the world's second-largest chipmaker after Intel (see Worldwide Top 20 Semiconductor Market Share Ranking Year by Year).[33]

1995: Samsung created its first liquid-crystal display screen

Source sentence: In 1995, it created its first liquid-crystal display screen. Ten years later, Samsung grew to be the world's largest manufacturer of liquid-crystal display panels.

1997: Samsung survived the Asian financial crisis relatively unharmed

Source sentence: Compared to other major Korean companies, Samsung survived the 1997 Asian financial crisis relatively unharmed.

1999: Samsung was founded as Korea Aerospace Industries (KAI)

Source sentence: The company was founded in 1999 as Korea Aerospace Industries (KAI), the result of a merger between then three domestic major aerospace divisions of Samsung Aerospace, Daewoo Heavy Industries and Hyundai Space and Aircraft Company.

2000: Samsung R&D opened a development center in Warsaw, Poland

Source sentence: In 2000, Samsung R&D opened a development center in Warsaw, Poland.[36]

2004: Toshiba Samsung Storage Technology Corporation (TSST) was formed

Source sentence: Toshiba Samsung Storage Technology Corporation (TSST) is a joint venture between Samsung Electronics and Toshiba of Japan which specialises in optical disc drive manufacturing. TSST was formed in 2004, and Toshiba owns 51 per cent of its stock, while Samsung owns the remaining 49 per cent.

2006: S-LCD was established as a joint venture between Samsung and Sony

Source sentence: In 2006, S-LCD was established as a joint venture between Samsung and Sony in order to provide a stable supply of LCD panels for both manufacturers.

2008: Samsung launched its smartphone platform

Source sentence: Its work began with set-top-box technology before moving into digital TV and smartphones. The smartphone platform was developed with partners, officially launched with the original Samsung Solstice[37] line of devices and other derivatives in 2008, which was later developed into Samsung Galaxy line of devices including Notes, Edge and other products.

2010: Samsung announced a ten-year growth strategy centered around five businesses

Source sentence: In 2010, Samsung[who?] announced a ten-year growth strategy centered around five businesses.[39]

2011: Samsung reported consolidated revenues of 220 trillion KRW ($172.5 billion)

Source sentence: In FY 2009, Samsung reported consolidated revenues of 220 trillion KRW ($172.5 billion). In FY 2010, Samsung reported consolidated revenues of 280 trillion KRW ($258 billion), and profits of 30 trillion KRW ($27.6 billion) based upon a KRW-USD exchange rate of 1,084.5 KRW per USD, the spot rate as of 19 August 2011.[update][67]

2012: Samsung became the world's largest mobile phone maker by unit sales

Source sentence: In first quarter of 2012, Samsung Electronics became the world's largest mobile phone maker by unit sales, overtaking Nokia, which had been the market leader since 1998.[41][42]

2015: Samsung was granted more U.S. patents than any other company

Source sentence: In 2015, Samsung has been granted more U.S. patents than any other company. The company received 7,679 utility patents through 11 December.[51]

2016: Samsung recalled all Galaxy Note 7 smartphones worldwide

Source sentence: However, in early September 2016, Samsung suspended sales of the phone and announced an informal recall. This occurred after some units of the phones had batteries with a defect that caused them to produce excessive heat, leading to fires and explosions. Samsung replaced the recalled units of the phones with a new version; however, it was later discovered that the new version of the Galaxy Note 7 also had the battery defect. Samsung recalled all Galaxy Note 7 smartphones worldwide on 10 October 2016, and permanently ended production of the phone the following day.[53][54][55]

2018: Samsung inaugurated the world's largest mobile manufacturing facility in Noida, India

Source sentence: In 2018, they inaugurated the world's largest mobile manufacturing facility in Noida, India, in the presence of Indian Prime Minister Narendra Modi and South Korean President Moon Jae-in.[56][57]

2022: Samsung's revenue was equal to 22.4% of South Korea's $1.67 trillion GDP

Source sentence: Samsung's revenue was equal to 22.4% of South Korea's $1.67 trillion GDP in 2022.[62]

2023: Samsung announced its decision to reduce the production of memory chips

Source sentence: In 2023, Samsung announced its decision to reduce the production of memory chips. This action is on account of the company's projected 96% decline in quarterly operating profit - a 600 million won decline from the 14 trillion won in 2022. The said drop can be attributed to the weak demand after COVID and a slowing global economy. Despite this decision, the company's shares increased by more than 4%.[58]

**New Information**:

**Name**: Samsung C&T Corporation

Source sentence: None

**Founders**: None

Source sentence: None

**Founded**: None

Source sentence: None

**Headquarters**: None

Source sentence: None

**Industry**: Various (Environment & Asset, Food Culture, Resort, Medical, Chemical, Aerospace, Defense, Technology, etc.)

Source sentence: page_content='Samsung C&T Corporation covers the three main sectors of Environment & Asset, Food Culture and Resort.

**Products**: Various (chemical products, medical equipment, microprocessors, plasma displays, steel, etc.)

Source sentence: None

**Services**: Various (medical services, research and development, manufacturing, sales, etc.)

Source sentence: None

**Revenue**: None

Source sentence: None

**Timeline**:

1978: Samsung Thales Co., Ltd. was established

Source sentence: Samsung's involvement was passed on to the Hanwha Group as part of the Techwin transaction.[109]

1989: Samsung BP Chemicals was established

Source sentence: Samsung BP Chemicals, headquartered in Ulsan, is a 49:51 joint venture between Samsung and the UK-based BP, which was established in 1989 to produce and supply high-value-added chemical products.[123][124][125]

1990: MEMC entered into a joint venture agreement to construct a silicon plant in Korea

Source sentence: MEMC's joint venture with Samsung Electronics Company, Ltd. In 1990, MEMC entered into a joint venture agreement to construct a silicon plant in Korea.[139]

1994: Samsung Medical Center was founded

Source sentence: Samsung Medical Center was founded on 9 November 1994, under the philosophy of "contributing to improving the nation's health through the best medical service, advanced medical research and development of outstanding medical personnel".[103]

1995: Stemco was established

Source sentence: Stemco is a joint venture established between Samsung Electro-Mechanics and Toray Industries in 1995.[131]

1997: S-1 was founded

Source sentence: S-1 was founded as Korea's first specialized security business in 1997 and has maintained its position at the top of industry with the consistent willingness to take on challenges.[118][119]

1998: Alpha Processor Inc. was established

Source sentence: Alpha Processor Inc. (API) was established in 1998 as a joint venture with U.S.-based Compaq, to enter the high-end microprocessor market.[113]

1999: Brooks Automation Asia Co., Ltd. was established

Source sentence: Brooks Automation Asia Co., Ltd. is a joint venture between Brooks Automation (70%) and Samsung[who?] (30%) which was established in 1999.[121]

2000: Global Steel Exchange was formed

Source sentence: Global Steel Exchange was a joint venture formed in 2000 between Samsung, the U.S.-based Cargill, the Switzerland-based Duferco Group, and the Luxembourg-based Tradearbed (now part of the ArcelorMittal), to handle their online buying and selling of steel.[115]

2001: Samsung Thomson-CSF Co., Ltd. was renamed Samsung Thales Co., Ltd.

Source sentence: Samsung's involvement was passed on to the Hanwha Group as part of the Techwin transaction.[109]

2003: Samtron became a brand of Samsung

Source sentence: Samtron was a subsidiary of Samsung until 1999 when it became independent. After that, it continued to make computer monitors and plasma displays until 2003, Samtron became Samsung when Samtron was a brand.[citation needed]

2004: SD Flex Co., Ltd. was founded

Source sentence: SD Flex Co., Ltd. was founded in October 2004 as a joint venture corporation by Samsung[who?] and DuPont, one of the world's largest chemical companies.[127]

2004: S-LCD Corporation was established

Source sentence: S-LCD Corporation was a joint venture between Samsung Electronics (50% plus one share) and the Japan-based Sony Corporation (50% minus one share) established in April 2004.[126]

2007: POSS – SLPC s.r.o. was founded

Source sentence: POSS – SLPC s.r.o. was founded in 2007 as a subsidiary of Samsung C & T Corporation, Samsung C & T Deutschland and the company POSCO.[122]

2008: Siltronic Samsung Wafer Pte. Ltd, was officially opened

Source sentence: Siltronic Samsung Wafer Pte. Ltd, is a joint venture between Samsung and wholly owned Wacker Chemie subsidiary Siltronic. It was officially opened in Singapore in June 2008.[129]

2008: SB LiMotive was founded

Source sentence: SB LiMotive is a 50:50 joint company of Robert Bosch GmbH (commonly known as Bosch) and Samsung SDI founded in June 2008.[130]

2010: Samsung Medical Center and Pfizer agreed to collaborate on research

Source sentence: In 2010, the Samsung Medical Center and pharmaceutical multinational Pfizer agreed to collaborate on research to identify the genomic mechanisms responsible for clinical outcomes in hepatocellular carcinoma.[106][107]

2011: MEMC Korea Company was established

Source sentence: MEMC's joint venture with Samsung Electronics Company, Ltd. In 1990, MEMC entered into a joint venture agreement to construct a silicon plant in Korea.[139]

2011: MEMC and Samsung formed a joint venture to build a polysilicon plant

Source sentence: In 2011, MEMC Electronic Materials Inc. and an affiliate of Korean conglomerate Samsung formed a joint venture to build a polysilicon plant.[130]

2016: Samsung General Chemicals was sold to Hanwha

Source sentence: Samsung General Chemicals was sold to Hanwha. Another chemical division was sold to Lotte Corporation in 2016.[111][112]

2016: Samsung advertised its Galaxy S7, S7 Edge, A5, A7, S8, S8 Plus and Note 8 devices as able to survive short immersion in water

Source sentence: However, after many user complaints about the devices having issues after water submersion, such as charger port corrosion. The ACC have officially labelled the fact these devices have 'water resistance' listed as a feature misleading and proceeded with the fine.

2016: Samsung was involved in a South Korean political scandal

Source sentence: The investigative team of special prosecutors looking into the 2016 South Korean political scandal announced that the Blue House received money from South Korea's four largest chaebols (Samsung, Hyundai Motor Group, SK Group and LG Group) to fund pro-government demonstrations by conservative and far-right organizations such as the Korean Parent Federation (KPF) and the Moms Brigade.

2022: Australia's competition and consumer commission fined Samsung AU$14 million

Source sentence: In Australia during 2022, Australia's competition and consumer commission fined Samsung AU$14 million.

In [278]:
context_prompt = """Your task is to output a JSON object.
Let's think step-by-step and follow the following instructions:

Step 1: Read the source sentences and remember to never modify these source
sentences.

Step 2: Extract Information: Extract information from the major sections:
"Name", "Founders", "Founded", "Headquarters", "Industry", "Products",
"Services", "Revenue", "Timeline" along with their source sentences.

Step 3: Create JSON Object: The keys in the JSON object should have both the 
information and the source sentence of the information.

Step 4: For "Timeline" this should be the format:
"Timeline": {{
  "<date>": {{
    "event": "<event_description>",
    "source_sentence": "<sentence_where_you_got_the_event_from>"
  }}
}}
Step 4: Date Format: Ensure that dates are in DD-MM-YYYY format.

Step 5: Data Types:Values which can be represented as integers should strictly
be of integer data types with only numbers representing them.

IMPORTANT: IF SOURCE SENTENCE IS NOT AVAILABLE THAT INFORMATION SHOULD NOT BE
PUT IN THE JSON OBJECT.
###
Context: {context}
###"""

In [279]:
prompt_template = ChatPromptTemplate.from_messages(
    [("user", context_prompt)]
)
llmchain = prompt_template | llm | parser
Markdown(llmchain.invoke({"context": context_samsung}))

Here is the JSON object based on the provided information:

```
{
    "Name": {
        "information": "Samsung",
        "source_sentence": "None (not mentioned in the provided chunk)"
    },
    "Founders": {
        "information": "Lee Byung-chul, Lee Kun-hee",
        "source_sentence": "Hotel Shilla (also known as \"The Shilla\") opened in March 1979, following the intention of the late Lee Byung-chul, founder of the Samsung Group.[102]"
    },
    "Founded": {
        "1985": {
            "event": "Samsung SDS was founded",
            "source_sentence": "Samsung SDS is a multinational IT Service company, founded in March 1985.[100]"
        },
        "1969": {
            "event": "Samsung Engineering was founded",
            "source_sentence": "Samsung Engineering is a multinational construction company headquartered in Seoul, founded in January 1969.[78]"
        },
        "1974": {
            "event": "Samsung Heavy Industries was founded",
            "source_sentence": "Samsung Heavy Industries is a shipbuilding and engineering company headquartered in Seoul, founded in August 1974.[83]"
        },
        "1994": {
            "event": "Samsung Medical Center was founded",
            "source_sentence": "Samsung Medical Center was founded on 9 November 1994, under the philosophy of \"contributing to improving the nation's health through the best medical service, advanced medical research and development of outstanding medical personnel\".[103]"
        },
        "1979": {
            "event": "Hotel Shilla was opened",
            "source_sentence": "Hotel Shilla (also known as \"The Shilla\") opened in March 1979, following the intention of the late Lee Byung-chul, founder of the Samsung Group.[102]"
        }
    },
    "Headquarters": {
        "information": "Suwon, South Korea (Samsung Electronics)",
        "source_sentence": "Samsung Electronics is a multinational electronics and information technology company headquartered in Suwon and the flagship company of the Samsung Group.[69]"
    },
    "Industry": {
        "information": "Electronics and information technology, construction, shipbuilding and engineering, general insurance, life insurance, biopharmaceuticals, Environment & Asset, Food Culture, Resort, Medical, Chemical, Aerospace, Defense, Technology, etc.",
        "source_sentence": "Samsung Electronics is a multinational electronics and information technology company headquartered in Suwon and the flagship company of the Samsung Group.[69]"
    },
    "Products": {
        "information": "Air conditioners, computers, digital television sets, active-matrix organic light-emitting diodes (AMOLEDs), mobile phones, display monitors, computer printers, refrigerators, semiconductors and telecommunications networking equipment, Chemical products, medical equipment, microprocessors, plasma displays, steel, etc.",
        "source_sentence": "Its products include air conditioners, computers, digital television sets, active-matrix organic light-emitting diodes (AMOLEDs), mobile phones, display monitors, computer printers, refrigerators, semiconductors and telecommunications networking equipment.[70]"
    },
    "Services": {
        "information": "IT services, insurance services, biopharmaceutical services, construction services, shipbuilding and engineering services, medical services, research and development, manufacturing, sales, etc.",
        "source_sentence": "Samsung SDS is a multinational IT Service company, founded in March 1985.[100]"
    },
    "Revenue": {
        "information": "$14 billion (U.S.) on advertising and marketing in 2013, Over 306 billion USD in export revenue to Vietnam from 2018 to 2022, 65 billion USD in export revenue to Vietnam in 2022",
        "source_sentence": "Samsung Electronics spent an estimated $14 billion (U.S.) on advertising and marketing in 2013. In the period from 2018 to 2022, Samsung contributed over 306 billion USD in export revenue to Vietnam. In 2022 alone, despite the impact of the COVID-19 pandemic, the figure reached 65 billion USD"
    },
    "Timeline": {
        "1980s-1990s": {
            "event": "Samsung manufactured a range of aircraft",
            "source_sentence": "The company was founded in 1999 as Korea Aerospace Industries (KAI), the result of a merger between then three domestic major aerospace divisions of Samsung Aerospace, Daewoo Heavy Industries and Hyundai Space and Aircraft Company. However, Samsung still manufactures aircraft engines and gas turbines.[35]"
        },
        "1987": {
            "event": "Samsung Group was separated into five business groups",
            "source_sentence": "After Lee, the founder's death in 1987, Samsung Group was separated into five business groups – Samsung Group, Shinsegae Group, CJ Group, Hansol Group and the JoongAng Group.[26]"
        },
        "1992": {
            "event": "Samsung became the world's largest producer of memory chips",
            "source_sentence": "Samsung became the world's largest producer of memory chips in 1992 and is the world's second-largest chipmaker after Intel (see Worldwide Top 20 Semiconductor Market Share Ranking Year by Year).[33]"
        },
        "1995": {
            "event": "Samsung created its first liquid-crystal display screen",
            "source_sentence": "In 1995, it created its first liquid-crystal display screen. Ten years later, Samsung grew to be the world's largest manufacturer of liquid-crystal display panels."
        },
        "1997": {
            "event": "Samsung survived the Asian financial crisis relatively unharmed",
            "source_sentence": "Compared to other major Korean companies, Samsung survived the 1997 Asian financial crisis relatively unharmed."
        },
        "1999": {
            "event": "Samsung was founded as Korea Aerospace Industries (KAI)",
            "source_sentence": "The company was founded in 1999 as Korea Aerospace Industries (KAI), the result of a merger between then three domestic major aerospace divisions of Samsung Aerospace, Daewoo Heavy Industries and Hyundai Space and Aircraft Company."
        },
        "2000": {
            "event": "Samsung R&D opened a development center in Warsaw, Poland",
            "source_sentence": "In 2000, Samsung R&D opened a development center in Warsaw, Poland.[36]"
        },
        "2004": {
            "event": "Toshiba Samsung Storage Technology Corporation (TSST) was formed",
            "source_sentence": "Toshiba Samsung Storage Technology Corporation (TSST) is a joint venture between Samsung Electronics and Toshiba of Japan which specialises in optical disc drive manufacturing. TSST was formed in 2004, and Toshiba owns 51 per cent of its stock, while Samsung owns the remaining 49 per cent."
        },
        "2006": {
            "event": "S-LCD was established as a joint venture between Samsung and Sony",
            "source_sentence": "In 2006, S-LCD was established as a joint venture between Samsung and Sony in order to provide a stable supply of LCD panels for both manufacturers."
        },
        "2008": {
            "event": "Samsung launched its smartphone platform",
            "source_sentence": "Its work began with set-top-box technology before moving into digital TV and smartphones. The smartphone platform was developed with partners, officially launched with the original Samsung Solstice[37] line of devices and other derivatives in 2008, which was later developed into Samsung Galaxy line of devices including Notes, Edge and other products."
        },
        "2010": {
            "event": "Samsung announced a ten-year growth strategy centered around five businesses",
            "source_sentence": "In 2010, Samsung[who?] announced a ten-year growth strategy centered around five businesses.[39]"
        },
        "2011": {
            "event": "Samsung reported consolidated revenues of 220 trillion KRW ($172.5 billion)",
            "source_sentence": "In FY 2009, Samsung reported consolidated revenues of 220 trillion KRW ($172.5 billion). In FY 2010, Samsung reported consolidated revenues of 280 trillion KRW ($258 billion), and profits of 30 trillion KRW ($27.6 billion) based upon a KRW-USD exchange rate of 1,084.5 KRW per USD, the spot rate as of 19 August 2011.[update][67]"
        },
        "2012": {
            "event": "Samsung became the world's largest mobile phone maker by unit sales",
            "source_sentence": "In first quarter of 2012, Samsung Electronics became the world's largest mobile phone maker by unit sales, overtaking Nokia, which had been the market leader since 1998.[41][42]"
        },
        "2015": {
            "event": "Samsung was granted more U.S. patents than any other company",
            "source_sentence": "In 2015, Samsung has been granted more U.S. patents than any other company. The company received 7,679 utility patents through 11 December.[51]"
        },
        "2016": {
            "event": "Samsung recalled all Galaxy Note 7 smartphones worldwide",
            "source_sentence": "However, in early September 2016, Samsung suspended sales of the phone and announced an informal recall. This occurred after some units of the phones had batteries with a defect that caused them to produce excessive heat, leading to fires and explosions. Samsung replaced the recalled units of the phones with a new version; however, it was later discovered that the new version of the Galaxy Note 7 also had the battery defect. Samsung recalled all Galaxy Note 7 smartphones worldwide on 10 October 2016, and permanently ended production of the phone the following day.[53][54][55]"
        },
        "2018": {
            "event": "Samsung inaugurated the world's largest mobile manufacturing facility in Noida, India",
            "source_sentence": "In 2018, they inaugurated the world's largest mobile manufacturing facility in Noida, India, in the presence of Indian Prime Minister Narendra Modi and South Korean President Moon Jae-in.[56][57]"
        },
        "2022": {
            "event": "Samsung's revenue was equal to 22.4% of South Korea's $1.67 trillion GDP",
            "source_sentence": "Samsung's revenue was equal to 22.4% of South Korea's $1.67 trillion GDP in 2022.[62]"
        },
        "2023": {
            "event": "Samsung announced its decision to reduce the production of memory chips",
            "source_sentence": "In 2023, Samsung announced its decision to reduce the production of memory chips. This action is on account of the company's projected 96% decline in quarterly operating profit - a 600 million won decline from the 14 trillion won in 2022. The said drop can be attributed to the weak demand after COVID and a slowing global economy. Despite this decision, the company's shares increased by more than 4%.[58]"
        },
        "1978": {
            "event": "Samsung Thales Co., Ltd. was established",
            "source_sentence": "Samsung's involvement was passed on to the Hanwha Group as part of the Techwin transaction.[109]"
        },
        "1989": {
            "event": "Samsung BP Chemicals was established",
            "source_sentence": "Samsung BP Chemicals, headquartered in Ulsan, is a 49:51 joint venture between Samsung and the UK-based BP, which was established in 1989 to produce and supply high-value-added chemical products.[123][124][125]"
        },
        "1990": {
            "event": "MEMC entered into a joint venture agreement to construct a silicon plant in Korea",
            "source_sentence": "MEMC's joint venture with Samsung Electronics Company, Ltd. In 1990, MEMC entered into a joint venture agreement to construct a silicon plant in Korea.[139]"
        },
        "1994": {
            "event": "Samsung Medical Center was founded",
            "source_sentence": "Samsung Medical Center was founded on 9 November 1994, under the philosophy of \"contributing to improving the nation's health through the best medical service, advanced medical research and development of outstanding medical personnel\".[103]"
        },
        "1995": {
            "event": "Stemco was established",
            "source_sentence": "Stemco is a joint venture established between Samsung Electro-Mechanics and Toray Industries in 1995.[131]"
        },
        "1997": {
            "event": "S-1 was founded",
            "source_sentence": "S-1 was founded as Korea's first specialized security business in 1997 and has maintained its position at the top of industry with the consistent willingness to take on challenges.[118][119]"
        },
        "1998": {
            "event": "Alpha Processor Inc. was established",
            "source_sentence": "Alpha Processor Inc. (API) was established in 1998 as a joint venture with U.S.-based Compaq, to enter the high-end microprocessor market.[113]"
        },
        "1999": {
            "event": "Brooks Automation Asia Co., Ltd. was established",
            "source_sentence": "Brooks Automation Asia Co., Ltd. is a joint venture between Brooks Automation (70%) and Samsung[who?] (30%) which was established in 1999.[121]"
        },
        "2000": {
            "event": "Global Steel Exchange was formed",
            "source_sentence": "Global Steel Exchange was a joint venture formed in 2000 between Samsung, the U.S.-based Cargill, the Switzerland-based Duferco Group, and the Luxembourg-based Tradearbed (now part of the ArcelorMittal), to handle their online buying and selling of steel.[115]"
        },
        "2001": {
            "event": "Samsung Thomson-CSF Co., Ltd. was renamed Samsung Thales Co., Ltd.",
            "source_sentence": "Samsung's involvement was passed on to the Hanwha Group as part of the Techwin transaction.[109]"
        },
        "2003": {
            "event": "Samtron became a brand of Samsung",
            "source_sentence": "Samtron was a subsidiary of Samsung until 1999 when it became independent. After that, it continued to make computer monitors and plasma displays until 2003, Samtron became Samsung when Samtron was a brand.[citation needed]"
        },
        "2004": {
            "event": "SD Flex Co., Ltd. was founded",
            "source_sentence": "SD Flex Co., Ltd. was founded in October 2004 as a joint venture corporation by Samsung[who?] and DuPont, one of the world's largest chemical companies.[127]"
        },
        "2004": {
            "event": "S-LCD Corporation was established",
            "source_sentence": "S-LCD Corporation was a joint venture between Samsung Electronics (50% plus one share) and the Japan-based Sony Corporation (50% minus one share) established in April 2004.[126]"
        },
        "2007": {
            "event": "POSS – SLPC s.r.o. was founded",
            "source_sentence": "POSS – SLPC s.r.o. was founded in 2007 as a subsidiary of Samsung C & T Corporation, Samsung C & T Deutschland and the company POSCO.[122]"
        },
        "2008": {
            "event": "Siltronic Samsung Wafer Pte. Ltd, was officially opened",
            "source_sentence": "Siltronic Samsung Wafer Pte. Ltd, is a joint venture between Samsung and wholly owned Wacker Chemie subsidiary Siltronic. It was officially opened in Singapore in June 2008.[129]"
        },
        "2008": {
            "event": "SB LiMotive was founded",
            "source_sentence": "SB LiMotive is a 50:50 joint company of Robert Bosch GmbH (commonly known as Bosch) and Samsung SDI founded in June 2008.[130]"
        },
        "2010": {
            "event": "Samsung Medical Center and Pfizer agreed to collaborate on research",
            "source_sentence": "In 2010, the Samsung Medical Center and pharmaceutical multinational Pfizer agreed to collaborate on research to identify the genomic mechanisms responsible for clinical outcomes in hepatocellular carcinoma.[106][107]"
        },
        "2011": {
            "event": "MEMC Korea Company was established",
            "source_sentence": "MEMC's joint venture with Samsung Electronics Company, Ltd. In 1990, MEMC entered into a joint venture agreement to construct a silicon plant in Korea.[139]"
        },
        "2011": {
            "event": "MEMC and Samsung formed a joint venture to build a polysilicon plant",
            "source_sentence": "In 2011, MEMC Electronic Materials Inc. and an affiliate of Korean conglomerate Samsung formed a joint venture to build a polysilicon plant.[130]"
        },
        "2016": {
            "event": "Samsung General Chemicals was sold to Hanwha",
            "source_sentence": "Samsung General Chemicals was sold to Hanwha. Another chemical division was sold to Lotte Corporation in 2016.[111][112]"
        },
        "2016": {
            "event": "Samsung advertised its Galaxy S7, S7 Edge, A5, A7, S8, S8 Plus and Note 8 devices as able to survive short immersion in water",
            "source_sentence": "However, after many user complaints about the devices having issues after water submersion, such as charger port corrosion. The ACC have officially labelled the fact these devices have 'water resistance' listed as a feature misleading and proceeded with the fine."
        },
        "2016": {
            "event": "Samsung was involved in a South Korean political scandal",
            "source_sentence": "The investigative team of special prosecutors looking into the 2016 South Korean political scandal announced that the Blue House received money from South Korea's four largest chaebols (Samsung, Hyundai Motor Group, SK Group and LG Group) to fund pro-government demonstrations by conservative and far-right organizations such as the Korean Parent Federation (KPF) and the Moms Brigade."
        },
        "2022": {
            "event": "Australia's competition and consumer commission fined Samsung AU$14 million",
            "source_sentence": "In Australia during 2022, Australia's competition and consumer commission fined Samsung AU$14 million."
    }
}
```
Please note that I have only included the information that is available in the provided text. If there is any additional information that you would like to include, please let me know and I will do my best to assist you.

In [158]:
# URL of the Wikipedia page you want to fetch
url = "https://en.wikipedia.org/wiki/Apple_Inc."

# Fetch the HTML content of the page
apple_html_content = fetch_wikipedia_page_by_url(url)

# Check if the content was fetched successfully and print it
if apple_html_content:
    print("Content fetched!")
else:
    print(f"Page at '{url}' not found or could not be fetched.")


Content fetched!


In [159]:
# Create an instance of RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000,
    chunk_overlap=2000
)

# Split the text into chunks and store the output
apple_split = splitter.create_documents([apple_html_content])

In [291]:
apple_responses = []
for i, chunk in enumerate(apple_split):
    apple_responses.append(process_chunk_with_llm(chunk, llm))

In [292]:
Markdown(apple_responses[0])

Here is the extracted information in the required format:

**Name**: Apple Inc.

Source sentence: page_content='Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley.

**Founders**: Steve Jobs, Steve Wozniak, and Ronald Wayne

Source sentence: Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership.

**Founded**: April 1, 1976

Source sentence: Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership.

**Headquarters**: Cupertino, California, in Silicon Valley

Source sentence: Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley.

**Industry**: Technology company

Source sentence: Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley.

**Products**: iPhone, iPad, Mac, Apple Watch, Vision Pro, Apple TV; software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS; and services like Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+

Source sentence: Apple's product lineup includes include the iPhone, iPad, Mac, Apple Watch, Vision Pro, and Apple TV; as well as software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS; and services like Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+.

**Services**: Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+

Source sentence: Apple's product lineup includes include the iPhone, iPad, Mac, Apple Watch, Vision Pro, and Apple TV; as well as software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS; and services like Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+.

**Revenue**: Not specified

Note: The revenue figures mentioned in the text are in billions of US dollars, but the exact revenue is not provided.

**Timeline**:
    1976: Apple Computer Company was founded on April 1, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership.
    1977: Apple Computer, Inc. was incorporated on January 3, without Wayne, who had left and sold his share of the company back to Jobs and Wozniak for $800.
    1979: VisiCalc, a spreadsheet program, was released for the Apple II.
    1980: Apple went public on December 12, selling 4.6 million shares at $22 per share.
    1981: A plane crash forced Wozniak to step back from the Macintosh project.
    1984: Apple launched the Macintosh on January 22, with a television advertisement directed by Ridley Scott.
    1985: Jobs was removed as the head of the Macintosh division, and he resigned from Apple in September.
    2022: Apple became the first publicly traded U.S. company to be valued at over $3 trillion.
    2024: Apple was still valued at over $3.2 trillion.

Note: The timeline only includes events mentioned in the provided chunk of context, and may not be a comprehensive timeline of Apple's history.

In [293]:
context_apple = ""
for response in apple_responses:
    context_apple = refine(response, context_apple)

In [294]:
len(context_apple)

12477

In [295]:
Markdown(context_apple)

Step 1: Read and Understand

I have read and understood the provided chunk and its source sentences.

Step 2: Analyze New Information

I have analyzed the new chunk of information and identified the important details:

* **Name**: Wingtech
* **Founders**: Not mentioned
* **Founded**: Not mentioned
* **Headquarters**: Not mentioned
* **Industry**: Electronics manufacturing
* **Products**: iPod, iPhone, iPad, circuit boards
* **Services**: Not mentioned
* **Revenue**: Not mentioned
* **Timeline**:

Step 3: Extract Information

I have extracted the important details if available:

* **Name**: Wingtech
* **Founders**: Not mentioned
* **Founded**: Not mentioned
* **Headquarters**: Not mentioned
* **Industry**: Not mentioned (but can be inferred as electronics manufacturing)
* **Products**: iPod, iPhone, iPad, circuit boards
* **Services**: Not mentioned
* **Revenue**: Not mentioned
* **Timeline**:

Step 4: Merge Information

I will merge the new information with the existing context, ensuring that no important detail is omitted. Here is the merged summary:

**Name**: Apple

Source sentence: None (not provided in the chunk)

**Founders**: Not mentioned

**Founded**: Not mentioned

**Headquarters**: Apple operates from the Apple Campus (also known by its address, 1 Infinite Loop), a grouping of six buildings in Cupertino that total 850,000 square feet (79,000 m2) located about 1 mile (1.6 km) to the west of Apple Park, and Cork, Ireland.

Source sentence: Apple operates from the Apple Campus (also known by its address, 1 Infinite Loop), a grouping of six buildings in Cupertino that total 850,000 square feet (79,000 m2) located about 1 mile (1.6 km) to the west of Apple Park. Apple's international sales and distribution arms operate out of the campus in Cork.

**Industry**: Technology

Source sentence: None (not provided in the chunk)

**Products**: Not mentioned

**Services**: Not mentioned

**Revenue**:

* Apple's revenue in Russia amounted to 85 billion rubles.

Source sentence: In Russia, in 2022, the company's revenue amounted to 85 billion rubles.[7]

**Timeline**:

* 1977: Apple occupied Stephens Creek Three as its headquarters.
Source sentence: In addition to Apple Park and the Apple Campus, Apple occupies an additional thirty office buildings scattered throughout the city of Cupertino, including three buildings as prior headquarters: Stephens Creek Three from 1977 to 1978, Bandley One from 1978 to 1982, and Mariani One from 1982 to 1993.
* 1978: Apple occupied Bandley One as its headquarters.
Source sentence: In addition to Apple Park and the Apple Campus, Apple occupies an additional thirty office buildings scattered throughout the city of Cupertino, including three buildings as prior headquarters: Stephens Creek Three from 1977 to 1978, Bandley One from 1978 to 1982, and Mariani One from 1982 to 1993.
* 1980: Apple opened its first location outside of the United States in Cork, Ireland.
Source sentence: Apple's international sales and distribution arms operate out of the campus in Cork.
* 1982: Apple occupied Mariani One as its headquarters.
Source sentence: In addition to Apple Park and the Apple Campus, Apple occupies an additional thirty office buildings scattered throughout the city of Cupertino, including three buildings as prior headquarters: Stephens Creek Three from 1977 to 1978, Bandley One from 1978 to 1982, and Mariani One from 1982 to 1993.
* 1993: Apple occupied the Apple Campus as its headquarters.
Source sentence: The Apple Campus was the company's headquarters from its opening in 1993, until the opening of Apple Park in 2017.
* 2010: Apple received a score of 52 points out of 100 from Climate Counts, a nonprofit organization dedicated to directing consumers toward the greenest companies.
Source sentence: Climate Counts' evaluation of Apple's environmental practices
* 2011: Apple's CEO Tim Cook defended his company's tax tactics at a Senate hearing.
Source sentence: On May 21, 2013, Apple CEO Tim Cook defended his company's tax tactics at a Senate hearing.
* 2012: Apple released a statement committing to ending its use of coal and shifting to 100% renewable clean energy.
Source sentence: Apple released a statement on April 17, 2012, committing to ending its use of coal and shifting to 100% renewable clean energy.
* 2013: Apple was using 100% renewable energy to power their data centers.
Source sentence: By 2013, Apple was using 100% renewable energy to power their data centers.
* 2016: Apple repatriated $285 billion to America, resulting in a $38 billion tax payment spread over the following eight years.
Source sentence: In 2016, Apple repatriated $285 billion to America, resulting in a $38 billion tax payment spread over the following eight years.
* 2017: Apple opened Apple Park.
Source sentence: The Apple Campus was the company's headquarters from its opening in 1993, until the opening of Apple Park in 2017.
* 2017: Apple donated $5 million to relief efforts for both Hurricane Irma and Hurricane Harvey, and for the 2017 Central Mexico earthquake.
Source sentence: In 2017 it donated $5 million to relief efforts for both Hurricane Irma and Hurricane Harvey, and for the 2017 Central Mexico earthquake.
* 2020: Apple announced its plan to become carbon neutral across its entire business, manufacturing supply chain, and product life cycle by 2030.
Source sentence: On July 21, 2020, Apple announced its plan to become carbon neutral across its entire business, manufacturing supply chain, and product life cycle by 2030.
* 2021: Apple started a $200 million fund to combat climate change by removing 1 million metric tons of carbon dioxide from the atmosphere each year.
Source sentence: In April 2021, Apple said that it had started a $200 million fund in order to combat climate change by removing 1 million metric tons of carbon dioxide from the atmosphere each year.

Newly added information:

* 2006: One complex of factories in Shenzhen, China that assembled the iPod and other items had over 200,000 workers living and working within it. Employees regularly worked more than 60 hours per week and made around $100 per month.
Source sentence: In 2006, one complex of factories in Shenzhen, China that assembled the iPod and other items had over 200,000 workers living and working within it. Employees regularly worked more than 60 hours per week and made around $100 per month.[392][393]
* 2007: Apple started yearly audits of all its suppliers regarding worker's rights, slowly raising standards and pruning suppliers that did not comply.
Source sentence: In 2007, Apple started yearly audits of all its suppliers regarding worker's rights, slowly raising standards and pruning suppliers that did not comply.[395]
* 2008: Apple became the first electronics manufacturer to eliminate all polyvinyl chloride (PVC) and brominated flame retardants (BFRs) in its complete product line.
Source sentence: Following further campaigns by Greenpeace,[354] in 2008, Apple became the first electronics manufacturer to eliminate all polyvinyl chloride (PVC) and brominated flame retardants (BFRs) in its complete product line.[355]
* 2009: Apple's iPhone 3GS was free of PVC, arsenic, and BFRs.
Source sentence: In June 2009, Apple's iPhone 3GS was free of PVC, arsenic, and BFRs.[356][361]
* 2011: Apple was featured in Greenpeace's Guide to Greener Electronics, which ranks electronics manufacturers on sustainability, climate and energy policy, and how "green" their products are.
Source sentence: In November 2011, Apple was featured in Greenpeace's Guide to Greener Electronics, which ranks electronics manufacturers on sustainability, climate and energy policy, and how "green" their products are.[363]
* 2012: Apple opened an iOS app development center in Bangalore and a maps development office for 4,000 staff in Hyderabad.
Source sentence: In May 2016, Apple opened an iOS app development center in Bangalore and a maps development office for 4,000 staff in Hyderabad.[385]
* 2015: UL's Zero Waste to Landfill validation started, certifying that all manufacturing waste is reused, recycled, composted, or converted into energy (when necessary).
Source sentence: The standard, which started in January 2015, certifies that all manufacturing waste is reused, recycled, composted, or converted into energy (when necessary).[347]
* 2016: Apple issued a US$1.5 billion green bond (climate bond), the first ever of its kind by a U.S. tech company.
Source sentence: In February 2016, Apple issued a US$1.5 billion green bond (climate bond), the first ever of its kind by a U.S. tech company.[366]
* 2017: Apple announced a $1 billion funding project for "advanced manufacturing" in the United States.
Source sentence: In May 2017, the company announced a $1 billion funding project for "advanced manufacturing" in the United States.[370]
* 2020: Apple scheduled the opening of its first physical outlet in India for 2021, while an online store is to be launched by the end of the year.
Source sentence: At the beginning of 2020, Tim Cook announced that Apple schedules the opening of its first physical outlet in India for 2021, while an online store is to be launched by the end of the year.[390]
* 2021: Apple uses components from 43 countries.
Source sentence: As of 2021[update], Apple uses components from 43 countries.[373]
* 2022: The NewClimate Institute, a German environmental policy think tank, published a survey evaluating the transparency and progress of the climate strategies and carbon neutrality pledges announced by 25 major companies in the United States, finding that Apple's carbon neutrality pledge and climate strategy was unsubstantiated and misleading.
Source sentence: In February 2022, the NewClimate Institute, a German environmental policy think tank, published a survey evaluating the transparency and progress of the climate strategies and carbon neutrality pledges announced by 25 major companies in the United States that found that Apple's carbon neutrality pledge and climate strategy was unsubstantiated and misleading.[350][351]
* 2022: Apple dropped leather from its product lines, citing the material's "significant carbon footprint".
Source sentence: In 2023, Apple announced it was dropping leather from its product lines, citing the material's "significant carbon footprint".[352]
* 2023: Wingtech was reported by The Wall Street Journal to gain an additional foothold in Apple's supply chain following protests at a Foxconn factory in the Zhengzhou Airport Economy Zone.
Source sentence: During the 2022 COVID-19 protests in China, Chinese state-owned company Wingtech was reported by The Wall Street Journal to gain an additional foothold in Apple's supply chain following protests at a Foxconn factory in the Zhengzhou Airport Economy Zone.[391]

Step 5: Maintain Natural Readability

The final summary reads naturally and maintains logical flow and consistency.

Step 6: Formatting

The final context with each piece of information followed by its source sentence is formatted as follows:

**Name**: Apple

Source sentence: None (not provided in the chunk)

**Founders**: Not mentioned

**Founded**: Not mentioned

**Headquarters**: Apple operates from the Apple Campus (also known by its address, 1 Infinite Loop), a grouping of six buildings in Cupertino that total 850,000 square feet (79,000 m2) located about 1 mile (1.6 km) to the west of Apple Park, and Cork, Ireland.

Source sentence: Apple operates from the Apple Campus (also known by its address, 1 Infinite Loop), a grouping of six buildings in Cupertino that total 850,000 square feet (79,000 m2) located about 1 mile (1.6 km) to the west of Apple Park. Apple's international sales and distribution arms operate out of the campus in Cork.

**Industry**: Technology

Source sentence: None (not provided in the chunk)

**Products**: Not mentioned

**Services**: Not mentioned

**Revenue**:

* Apple's revenue in Russia amounted to 85 billion rubles.

Source sentence: In Russia, in 2022, the company's revenue amounted to 85 billion rubles.[7]

**Timeline**:

* 1977: Apple occupied Stephens Creek Three as its headquarters.

Source sentence: In addition to Apple Park and the Apple Campus, Apple occupies an additional thirty office buildings scattered throughout the city of Cupertino, including three buildings as prior headquarters: Stephens Creek Three from 1977 to 1978, Bandley One from 1978 to 1982, and Mariani One from 1982 to 1993.

...and so on.

In [173]:
prompt_template = ChatPromptTemplate.from_messages(
    [("user", context_prompt)]
)
llmchain = prompt_template | llm | parser
Markdown(llmchain.invoke({"context": context_apple}))

Here is the JSON object:

```
{
  "Name": {
    "information": "Apple",
    "source_sentence": "Apple announced on August 16, 2016, that Lens Technology, one of its major suppliers in China, has committed to power all its glass production for Apple with 100 percent renewable energy by 2018."
  },
  "Founders": {
    "information": "Steve Jobs, Steve Wozniak",
    "source_sentence": "Steve Jobs and Steve Wozniak were fans of the Beatles... In a 2017 interview, Wozniak credited watching Star Trek and attending Star Trek conventions in his youth as inspiration for co-founding Apple."
  },
  "Founded": {
    "information": 1976,
    "source_sentence": "Apple was founded as Apple Computer Company on April 1, 1976, to produce and market Steve Wozniak's Apple I personal computer."
  },
  "Headquarters": {
    "information": "Cupertino, California",
    "source_sentence": "Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley. Apple Inc.'s world corporate headquarters are located in Cupertino, in the middle of California's Silicon Valley, at Apple Park, a massive circular groundscraper building with a circumference of one mile (1.6 km)."
  },
  "Industry": {
    "information": "Technology",
    "source_sentence": "Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley."
  },
  "Products/Services": {
    "information": "iPhone, iPad, Mac, Apple Watch, Vision Pro, Apple TV, software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS, and services like Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+",
    "source_sentence": "Apple's product lineup includes include the iPhone, iPad, Mac, Apple Watch, Vision Pro, and Apple TV; as well as software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS; and services like Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+."
  },
  "Revenue": {
    "information": 394300000000,
    "source_sentence": "In 2022, Apple was the largest technology company by revenue, with US$394.3 billion."
  },
  "Timeline": {
    "16-08-1968": {
      "event": "Apple Corps Ltd. was founded",
      "source_sentence": "Apple Corps Ltd., a multimedia company started by the Beatles in 1968."
    },
    "01-04-1976": {
      "event": "Apple was founded as Apple Computer Company",
      "source_sentence": "Apple was founded as Apple Computer Company on April 1, 1976, to produce and market Steve Wozniak's Apple I personal computer."
    },
    "03-01-1977": {
      "event": "Apple Computer, Inc. was incorporated",
      "source_sentence": "Apple Computer, Inc. was incorporated on January 3, 1977, without Wayne, who had left and sold his share of the company back to Jobs and Wozniak for $800 only twelve days after having co-founded it."
    },
    "2006": {
      "event": "One complex of factories in Shenzhen, China that assembled the iPod and other items had over 200,000 workers living and working within it, and employees regularly worked more than 60 hours per week and made around $100 per month",
      "source_sentence": "In 2006, one complex of factories in Shenzhen, China that assembled the iPod and other items had over 200,000 workers living and working within it. Employees regularly worked more than 60 hours per week and made around $100 per month."
    },
    "2007": {
      "event": "Apple started yearly audits of all its suppliers regarding worker's rights",
      "source_sentence": "In 2007, Apple started yearly audits of all its suppliers regarding worker's rights, slowly raising standards and pruning suppliers that did not comply."
    },
    "2008": {
      "event": "Apple eliminated all polyvinyl chloride (PVC) and brominated flame retardants (BFRs) in its complete product line",
      "source_sentence": "Following further campaigns by Greenpeace, in 2008, Apple became the first electronics manufacturer to eliminate all polyvinyl chloride (PVC) and brominated flame retardants (BFRs) in its complete product line."
    },
    "2011": {
      "event": "Apple's profit margins were 40 percent",
      "source_sentence": "The company's manufacturing, procurement, and logistics enable it to execute massive product launches without having to maintain large, profit-sapping inventories. In 2011, Apple's profit margins were 40 percent, compared with between 10 and 20 percent for most other hardware companies."
    },
    "2012": {
      "event": "Apple opened an iOS app development center in Bangalore and a maps development office for 4,000 staff in Hyderabad",
      "source_sentence": "In May 2016, Apple opened an iOS app development center in Bangalore and a maps development office for 4,000 staff in Hyderabad."
    },
    "2015": {
      "event": "UL's Zero Waste to Landfill validation standard was started",
      "source_sentence": "Apple also announced that all 14 of its final assembly sites in China are now compliant with UL's Zero Waste to Landfill validation. The standard, which started in January 2015, certifies that all manufacturing waste is reused, recycled, composted, or converted into energy (when necessary)."
    },
    "2016": {
      "event": "Apple issued a US$1.5 billion green bond (climate bond)",
      "source_sentence": "In February 2016, Apple issued a US$1.5 billion green bond (climate bond), the first ever of its kind by a U.S. tech company."
    },
    "2017": {
      "event": "Apple announced a $1 billion funding project for 'advanced manufacturing' in the United States",
      "source_sentence": "In May 2017, the company announced a $1 billion funding project for 'advanced manufacturing' in the United States,"
    },
    "2020": {
      "event": "Apple announced plans to become carbon neutral across its entire business, manufacturing supply chain, and product life cycle by 2030",
      "source_sentence": "On July 21, 2020, Apple announced its plan to become carbon neutral across its entire business, manufacturing supply chain, and product life cycle by 2030."
    },
    "2021": {
      "event": "Apple's revenue in India was reported to be $1 billion",
      "source_sentence": "At the beginning of 2020, Tim Cook announced that Apple schedules the opening of its first physical outlet in India for 2021, while an online store is to be launched by the end of the year."
    },
    "2022": {
      "event": "Apple's revenue in Russia was reported to be 85 billion rubles",
      "source_sentence": "In Russia, in 2022, the company's revenue amounted to 85 billion rubles."
    }
  },
  "Name": {
    "information": "Wingtech",
    "source_sentence": "During the 2022 COVID-19 protests in China, Chinese state-owned company Wingtech was reported by The Wall Street Journal to gain an additional foothold in Apple's supply chain following protests at a Foxconn factory in the Zhengzhou Airport Economy Zone."
  },
  "Industry": {
    "information": "Contract electronics manufacturer",
    "source_sentence": "The Foxconn (Chinese: 富士康) employees attempted suicide,[397] resulting in 14 deaths—the company was the world's largest contract electronics manufacturer, for clients including Apple, at the time."
  },
  "Timeline": {
    "2006": {
      "event": "One complex of factories in Shenzhen, China that assembled the iPod and other items had over 200,000 workers living and working within it, and employees regularly worked more than 60 hours per week and made around $100 per month",
      "source_sentence": "In 2006, one complex of factories in Shenzhen, China that assembled the iPod and other items had over 200,000 workers living and working within it. Employees regularly worked more than 60 hours per week and made around $100 per month."
    },
    "2007": {
      "event": "Apple started yearly audits of all its suppliers regarding worker's rights",
      "source_sentence": "In 2007, Apple started yearly audits of all its suppliers regarding worker's rights, slowly raising standards and pruning suppliers that did not comply."
    },
    "2010": {
      "event": "Foxconn suicides occurred, with 18 employees attempting suicide and 14 deaths",
      "source_sentence": "The Foxconn suicides occurred between January and November 2010, when 18 Foxconn (Chinese: 富士康) employees attempted suicide,[397] resulting in 14 deaths—the company was the world's largest contract electronics manufacturer, for clients including Apple, at the time."
    },
    "2011": {
      "event": "Apple admitted that its suppliers' child labor practices in China had worsened",
      "source_sentence": "In 2011, Apple admitted that its suppliers' child labor practices in China had worsened."
    },
    "2014": {
      "event": "A BBC investigation found excessive hours and other problems persisted, and the Institute for Global Labour and Human Rights published a report documenting inhumane conditions for workers at a Zhen Ding Technology factory",
      "source_sentence": "A 2014 BBC investigation found excessive hours and other problems persisted, after Apple's promise to reform factory practice after the 2010 Foxconn suicides."
    },
    "2019": {
      "event": "Reports stated that some of Foxconn's managers had used rejected parts to build iPhones and that Apple was investigating the issue",
      "source_sentence": "In 2019, there were reports stating that some of Foxconn's managers had used rejected parts to build iPhones and that Apple was investigating the issue."
    }
  }
}
```

In [316]:
refine_prompt_2 = """Your job is to add the new important details to the given
context from the given chunk. 
The details you should keep track of are; "Name", "Founders", "Founded",
"Headquarters", "Industry", "Products", "Services", "Revenue", "Timeline".
For each piece of information in the chunk there will be a source sentence 
relating to that piece of information.
IMPORTANT: THE SOURCE SENTENCES SHOULD ALWAYS BE COPIED WORD FOR WORD.
Let's think step-by-step.
Step 1: Examine the context: Determine what information the context already has
.You do not need to change this information, you only need to add to it if
possible.
Step 2: Examine the chunk: Carefully read the provided chunk and its source
sentence to understand all the important points.
Step 2: Analyze New Information: Analyze the new chunk of information and
identify important informations and their respective source sentences.
Step 3: Extract Information: Note down these important details if available:
"Name", "Founders", "Founded", "Headquarters", "Industry", "Products",
"Services", "Revenue", "Timeline".
Step 4: Merge Information: Add the new information with the existing context, 
ensuring that no important detail is omitted. Be as detailed as possible while 
maintaining logical flow and consistency.
Step 5: Maintain Natural Readability: Ensure the final summary reads naturally,
maintains logical flow and consistency, and avoids repetition.
Step 6: Formatting: Present the final context with each piece of information 
followed by its source sentence, formatted as shown below. Source sentence
should always be included in the output and not referenced:
<Information>: <detail>
    Source sentence: <respective_source_sentence_of_the_information>
IMPORTANT: DO NOT FORGET THE IMPORTANT INFORMATION THAT IS ALREADY PRESENT IN
THE CONTEXT. ONLY ADD NEW RELEVANT INFORMATION FROM THE CHUNK TO THE OLD 
CONTEXT
###
Context: {context}
###
###
Chunk: {chunk}
###"""

In [317]:
def refine_2(chunk, context):
    """
    Refines and merges important details from a given context and a new chunk 
    of information to create a comprehensive and coherent summary.
    The function processes the context and the new chunk of information using a
    language model (LLM) and returns a detailed and accurate summary that 
    includes all critical points.

    Parameters:
    chunk (str): New chunk of information to be integrated with the context.
    context (str): Existing context that provides background or previous
    information.

    Returns:
    str: A refined summary that integrates information from both the context 
    and the new chunk.
    """
    prompt_template = ChatPromptTemplate.from_messages(
        [("user", refine_prompt_2)]
    )
    llmchain = prompt_template | llm | parser
    return llmchain.invoke({"context": context, "chunk": chunk})

In [318]:
context_apple_2 = ""
for response in apple_responses:
    context_apple_2 = refine_2(response, context_apple_2)
    print(context_apple_2)
    print("="*100)

Step 1: Examine the context
The context already has the following information:
- Name: Apple Inc.
- Founders: Steve Jobs, Steve Wozniak, and Ronald Wayne
- Founded: April 1, 1976
- Headquarters: Cupertino, California, in Silicon Valley
- Industry: Technology company
- Products: iPhone, iPad, Mac, Apple Watch, Vision Pro, Apple TV; software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS; and services like Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+
- Services: Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+
- Revenue: Not specified
- Timeline: A brief timeline of Apple's history, including its founding, going public, and major events.

Step 2: Examine the chunk
The chunk provides more information about Apple's products, services, and timeline.

Step 3: Analyze New Information
The new information includes:
- No new Name, Founders, or Headquarters information
- No new Industry information
- No new Services information
- No new Revenue information
- The timeli

In [319]:
len(context_apple_2)

6646

In [320]:
Markdown(context_apple_2)

Based on the provided context and chunk, I will add the new information to the existing context. Here is the merged information:

**Name**: Apple Inc.

    Source sentence: Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley.

**Founders**: Steve Jobs, Steve Wozniak, and Ronald Wayne

    Source sentence: Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership.

**Founded**: April 1, 1976

    Source sentence: Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership.

**Headquarters**: Cupertino, California, in Silicon Valley, and Cork, Ireland

    Source sentence: Apple Inc.'s world corporate headquarters are located in Cupertino, in the middle of California's Silicon Valley, at Apple Park, a massive circular groundscraper building with a circumference of one mile (1.6 km). Apple's headquarters for Europe, the Middle East and Africa (EMEA) are located in Cork in the south of Ireland, called the Hollyhill campus.

**Industry**: Technology

    Source sentence: Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley.

**Products**: iPhone, iPad, Mac, Apple Watch, Vision Pro, Apple TV; software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS; and services like Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+

    Source sentence: Apple's product lineup includes include the iPhone, iPad, Mac, Apple Watch, Vision Pro, and Apple TV; as well as software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS; and services like Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+.

**Services**: Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+

    Source sentence: Apple's product lineup includes include the iPhone, iPad, Mac, Apple Watch, Vision Pro, and Apple TV; as well as software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS; and services like Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+.

**Revenue**: $108 billion (2011)

    Source sentence: In its fiscal year ending in September 2011, Apple Inc. reported a total of $108 billion in annual revenues—a significant increase from its 2010 revenues of $65 billion—and nearly $82 billion in cash reserves.

**Timeline**:
    ... (existing timeline)
    2006: One complex of factories in Shenzhen, China that assembled the iPod and other items had over 200,000 workers living and working within it. Employees regularly worked more than 60 hours per week and made around $100 per month.
        Source sentence: In 2006, one complex of factories in Shenzhen, China that assembled the iPod and other items had over 200,000 workers living and working within it. Employees regularly worked more than 60 hours per week and made around $100 per month.[392][393]
    ...
    * 2022: Apple's revenue in Russia amounted to 85 billion rubles.
        Source sentence: In Russia, in 2022, the company's revenue amounted to 85 billion rubles.[7]
    * 2022: Wingtech was reported by The Wall Street Journal to gain an additional foothold in Apple's supply chain following protests at a Foxconn factory in the Zhengzhou Airport Economy Zone.
        Source sentence: During the 2022 COVID-19 protests in China, Chinese state-owned company Wingtech was reported by The Wall Street Journal to gain an additional foothold in Apple's supply chain following protests at a Foxconn factory in the Zhengzhou Airport Economy Zone.[391]
    * 2023: Apple dropped leather from its product lines, citing the material's "significant carbon footprint".
        Source sentence: In 2023, Apple announced it was dropping leather from its product lines, citing the material's "significant carbon footprint".[352]

**New Information:**

**Name**: Wingtech

    Source sentence: During the 2022 COVID-19 protests in China, Chinese state-owned company Wingtech was reported by The Wall Street Journal to gain an additional foothold in Apple's supply chain following protests at a Foxconn factory in the Zhengzhou Airport Economy Zone.[391]

**Products**: iPod, iPhone, iPad, circuit boards

    Source sentence: In 2006, one complex of factories in Shenzhen, China that assembled the iPod and other items had over 200,000 workers living and working within it.

**Timeline:**

    2006: Apple launched an investigation and worked with their manufacturers to ensure acceptable working conditions. Yearly audits of all its suppliers regarding worker's rights started.
        Source sentence: Apple immediately launched an investigation after the 2006 media report, and worked with their manufacturers to ensure acceptable working conditions.[394] In 2007, Apple started yearly audits of all its suppliers regarding worker's rights, slowly raising standards and pruning suppliers that did not comply. Yearly progress reports have been published as of 2008[update].[395]

    2010: Foxconn suicides occurred, Apple admitted that its suppliers' child labor practices in China had worsened, and workers planned to sue iPhone contractors over poisoning by a cleaner used to clean LCD screens.
        Source sentence: The Foxconn suicides occurred between January and November 2010, when 18 Foxconn (Chinese: 富士康) employees attempted suicide,[397] resulting in 14 deaths—the company was the world's largest contract electronics manufacturer, for clients including Apple, at the time.[397][398][399]

    2011: Apple admitted that its suppliers' child labor practices in China had worsened.
        Source sentence: In 2011, Apple admitted that its suppliers' child labor practices in China had worsened.[396]

    2014: A BBC investigation found excessive hours and other problems persisted, and the Institute for Global Labour and Human Rights published a report documenting inhumane conditions for workers at a Zhen Ding Technology factory in Shenzhen, China.
        Source sentence: In 2014, the Institute for Global Labour and Human Rights published a report which documented inhumane conditions for the 15,000 workers at a Zhen Ding Technology factory in Shenzhen, China, which serves as a major supplier of circuit boards for Apple's iPhone and iPad.[407]

    2019: Reports stated that some of Foxconn's managers had used rejected parts to build iPhones, and Apple was investigating the issue.
        Source sentence: In 2019, there were reports stating that some of Foxconn's managers had used rejected parts to build iPhones and that Apple was investigating the issue.[408]

In [321]:
prompt_template = ChatPromptTemplate.from_messages(
    [("user", context_prompt)]
)
llmchain = prompt_template | llm | parser
Markdown(llmchain.invoke({"context": context_apple_2}))

Here is the resulting JSON object:

```
{
  "Name": {
    "information": "Apple Inc.",
    "source_sentence": "Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley."
  },
  "Founders": {
    "information": "Steve Jobs, Steve Wozniak, and Ronald Wayne",
    "source_sentence": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership."
  },
  "Founded": {
    "information": "April 1, 1976",
    "source_sentence": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership."
  },
  "Headquarters": {
    "information": "Cupertino, California, in Silicon Valley, and Cork, Ireland",
    "source_sentence": "Apple Inc.'s world corporate headquarters are located in Cupertino, in the middle of California's Silicon Valley, at Apple Park, a massive circular groundscraper building with a circumference of one mile (1.6 km). Apple's headquarters for Europe, the Middle East and Africa (EMEA) are located in Cork in the south of Ireland, called the Hollyhill campus."
  },
  "Industry": {
    "information": "Technology",
    "source_sentence": "Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley."
  },
  "Products": {
    "information": "iPhone, iPad, Mac, Apple Watch, Vision Pro, Apple TV; software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS; and services like Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+",
    "source_sentence": "Apple's product lineup includes include the iPhone, iPad, Mac, Apple Watch, Vision Pro, and Apple TV; as well as software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS; and services like Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+."
  },
  "Services": {
    "information": "Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+",
    "source_sentence": "Apple's product lineup includes include the iPhone, iPad, Mac, Apple Watch, Vision Pro, and Apple TV; as well as software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS; and services like Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+."
  },
  "Revenue": {
    "information": "108 billion (2011)",
    "source_sentence": "In its fiscal year ending in September 2011, Apple Inc. reported a total of $108 billion in annual revenues—a significant increase from its 2010 revenues of $65 billion—and nearly $82 billion in cash reserves."
  },
  "Timeline": {
    "2006": {
      "event": "One complex of factories in Shenzhen, China that assembled the iPod and other items had over 200,000 workers living and working within it. Employees regularly worked more than 60 hours per week and made around $100 per month.",
      "source_sentence": "In 2006, one complex of factories in Shenzhen, China that assembled the iPod and other items had over 200,000 workers living and working within it. Employees regularly worked more than 60 hours per week and made around $100 per month."
    },
    "2007": {
      "event": "Apple started yearly audits of all its suppliers regarding worker's rights, slowly raising standards and pruning suppliers that did not comply. Yearly progress reports have been published as of 2008.",
      "source_sentence": "In 2007, Apple started yearly audits of all its suppliers regarding worker's rights, slowly raising standards and pruning suppliers that did not comply. Yearly progress reports have been published as of 2008"
    },
    "2010": {
      "event": "Foxconn suicides occurred, Apple admitted that its suppliers' child labor practices in China had worsened, and workers planned to sue iPhone contractors over poisoning by a cleaner used to clean LCD screens.",
      "source_sentence": "The Foxconn suicides occurred between January and November 2010, when 18 Foxconn (Chinese: 富士康) employees attempted suicide,[397] resulting in 14 deaths—the company was the world's largest contract electronics manufacturer, for clients including Apple, at the time.[397][398][399]"
    },
    "2011": {
      "event": "Apple admitted that its suppliers' child labor practices in China had worsened.",
      "source_sentence": "In 2011, Apple admitted that its suppliers' child labor practices in China had worsened."
    },
    "2014": {
      "event": "A BBC investigation found excessive hours and other problems persisted, and the Institute for Global Labour and Human Rights published a report documenting inhumane conditions for workers at a Zhen Ding Technology factory in Shenzhen, China.",
      "source_sentence": "In 2014, the Institute for Global Labour and Human Rights published a report which documented inhumane conditions for the 15,000 workers at a Zhen Ding Technology factory in Shenzhen, China, which serves as a major supplier of circuit boards for Apple's iPhone and iPad.[407]"
    },
    "2019": {
      "event": "Reports stated that some of Foxconn's managers had used rejected parts to build iPhones, and Apple was investigating the issue.",
      "source_sentence": "In 2019, there were reports stating that some of Foxconn's managers had used rejected parts to build iPhones and that Apple was investigating the issue.[408]"
    },
    "2022": {
      "event": "Apple's revenue in Russia amounted to 85 billion rubles, and Wingtech was reported by The Wall Street Journal to gain an additional foothold in Apple's supply chain following protests at a Foxconn factory in the Zhengzhou Airport Economy Zone.",
      "source_sentence": "In Russia, in 2022, the company's revenue amounted to 85 billion rubles.[7] During the 2022 COVID-19 protests in China, Chinese state-owned company Wingtech was reported by The Wall Street Journal to gain an additional foothold in Apple's supply chain following protests at a Foxconn factory in the Zhengzhou Airport Economy Zone.[391]"
    },
    "2023": {
      "event": "Apple dropped leather from its product lines, citing the material's 'significant carbon footprint'.",
      "source_sentence": "In 2023, Apple announced it was dropping leather from its product lines, citing the material's 'significant carbon footprint'.[352]"
  }
}
```

Note: I followed the instructions to ensure that the JSON object only includes information where a source sentence is available.

In [322]:
context_samsung_2 = ""
for response in samsung_responses:
    context_samsung_2 = refine_2(response, context_samsung_2)

In [323]:
Markdown(context_samsung_2)

Here is the merged context with the new information:

Samsung Group is a South Korean multinational manufacturing conglomerate headquartered in Samsung Digital City, Suwon, South Korea. It was founded by Lee Byung-chul in 1938 as a trading company.

**Name**: Samsung Group
Source sentence: Samsung Group is a South Korean multinational manufacturing conglomerate headquartered in Samsung Digital City, Suwon, South Korea.[1]

**Founders**: Lee Byung-chul
Source sentence: Samsung was founded by Lee Byung-chul in 1938 as a trading company.

**Founded**: 1938
Source sentence: Samsung was founded by Lee Byung-chul in 1938 as a trading company.

**Headquarters**: Samsung Electronics is headquartered in Suwon, Samsung Group is headquartered in Suwon
Source sentence: Samsung Electronics is a multinational electronics and information technology company headquartered in Suwon and the flagship company of the Samsung Group.[69]

**Industry**: Electronics and information technology
Source sentence: Samsung Electronics is a multinational electronics and information technology company headquartered in Suwon and the flagship company of the Samsung Group.[69]

**Products**: Air conditioners, computers, digital television sets, active-matrix organic light-emitting diodes (AMOLEDs), mobile phones, display monitors, computer printers, refrigerators, semiconductors, and telecommunications networking equipment
Source sentence: Its products include air conditioners, computers, digital television sets, active-matrix organic light-emitting diodes (AMOLEDs), mobile phones, display monitors, computer printers, refrigerators, semiconductors and telecommunications networking equipment.[70]

**Services**: Financial services, shipbuilding, medical services, and research and development
Source sentence: Its activities include construction, consumer electronics, financial services, shipbuilding, and medical services,[64] and two research and development stations that have allowed the chaebol to enter the industries of "high-polymer chemicals, genetic engineering tools [and biotech as a whole], aerospace, and nanotechnology."[65]

**Revenue**: Over 306 billion USD in export revenue to Vietnam from 2018 to 2022, 65 billion USD in export revenue to Vietnam in 2022, $14 billion (U.S.) on advertising and marketing in 2013
Source sentences:
* "In the period from 2018 to 2022, Samsung contributed over 306 billion USD in export revenue to Vietnam."
* "In 2022 alone, despite the impact of the COVID-19 pandemic, the figure reached 65 billion USD"
* "Samsung Electronics spent an estimated $14 billion (U.S.) on advertising and marketing in 2013."

**Timeline**:
1980s-1990s: Samsung manufactured a range of aircraft
Source sentence: In the 1980s-1990s, Samsung manufactured a range of aircraft.
1982: Samsung Electronics built a television assembly plant in Portugal
Source sentence: In the 1980s, Samsung Electronics began to invest heavily in research and development, investments that were pivotal in pushing the company to the forefront of the global electronics industry. In 1982, it built a television assembly plant in Portugal; in 1984, a plant in New York; in 1985, a plant in Tokyo; in 1987, a facility in England; and another facility in Austin, Texas, in 1996.
1984: Samsung Electronics built a plant in New York
Source sentence: In the 1980s, Samsung Electronics began to invest heavily in research and development, investments that were pivotal in pushing the company to the forefront of the global electronics industry. In 1982, it built a television assembly plant in Portugal; in 1984, a plant in New York; in 1985, a plant in Tokyo; in 1987, a facility in England; and another facility in Austin, Texas, in 1996.
1987: Samsung started to globalize its activities and electronics
Source sentence: Since 1990, Samsung has increasingly globalised its activities and electronics; in particular, its mobile phones and semiconductors have become its most important source of income.
1987: Samsung Group was separated into five business groups
Source sentence: After Lee, the founder's death in 1987, Samsung Group was separated into five business groups – Samsung Group, Shinsegae Group, CJ Group, Hansol Group and the JoongAng Group.
1990: Samsung started to globalize its activities and electronics
Source sentence: Since 1990, Samsung has increasingly globalised its activities and electronics; in particular, its mobile phones and semiconductors have become its most important source of income.
1992: Samsung became the world's largest producer of memory chips
Source sentence: Samsung became the world's largest producer of memory chips in 1992 and is the world's second-largest chipmaker after Intel (see Worldwide Top 20 Semiconductor Market Share Ranking Year by Year).[33]
1995: Samsung created its first liquid-crystal display screen
Source sentence: In 1995, it created its first liquid-crystal display screen. Ten years later, Samsung grew to be the world's largest manufacturer of liquid-crystal display panels.
2000: Samsung R&D opened a development center in Warsaw, Poland
Source sentence: In 2000, Samsung R&D opened a development center in Warsaw, Poland.[36]
2004: Toshiba Samsung Storage Technology Corporation (TSST) was formed
Source sentence: In 2004, Toshiba Samsung Storage Technology Corporation (TSST) was formed.
2006: S-LCD was established as a joint venture between Samsung and Sony
Source sentence: In 2006, S-LCD was established as a joint venture between Samsung and Sony in order to provide a stable supply of LCD panels for both manufacturers. S-LCD was owned by Samsung (50% plus one share) and Sony (50% minus one share) and operates its factories and facilities in Tanjung, South Korea.
2022: Samsung's revenue was equal to 22.4% of South Korea's $1.67 trillion GDP
Source sentence: Samsung's revenue was equal to 22.4% of South Korea's $1.67 trillion GDP in 2022.[62]

**Additional Information**:

* **Name**: Samsung Group
Source sentence: As of April 2011, the Samsung Group comprised 59 unlisted companies and 19 listed companies, all of which had their primary listing on the Korea Exchange.[66]
* **Founders**: Lee Byung-chul
Source sentence: Hotel Shilla (also known as "The Shilla") opened in March 1979, following the intention of the late Lee Byung-chul, founder of the Samsung Group.[102]
* **Founded**: 1952 (Samsung Fire & Marine Insurance), 1969 (Samsung Engineering), 1974 (Samsung Heavy Industries), 1985 (Samsung SDS), 1987 (Samsung Advanced Institute of Technology), 1994 (Samsung Medical Center)
Source sentences:
    - Samsung Fire & Marine Insurance was founded in January 1952 as Korea Anbo Fire and Marine Insurance and was renamed Samsung Fire & Marine Insurance in December 1993.[81]
    - Samsung Engineering is a multinational construction company headquartered in Seoul, founded in January 1969.[78]
    - Samsung Heavy Industries is a shipbuilding and engineering company headquartered in Seoul, founded in August 1974.[83]
    - Samsung SDS is a multinational IT Service company, founded in March 1985.[100]
    - Samsung Advanced Institute of Technology (SAIT), established in 1987, is headquartered in Suwon and operates research labs around the world.[100]
    - Samsung Medical Center was founded on 9 November 1994, under the philosophy of "contributing to improving the nation's health through the best medical service, advanced medical research and development of outstanding medical personnel".[103]

* **Headquarters**: Suwon (Samsung Electronics), Seoul (Samsung Engineering, Samsung Heavy Industries, Samsung Fire & Marine Insurance, Samsung Life Insurance, Samsung C&T Corporation), Incheon (Samsung Biologics)
Source sentences:
    - Samsung Electronics is a multinational electronics and information technology company headquartered in Suwon and the flagship company of the Samsung Group.[69]
    - Samsung Engineering is a multinational construction company headquartered in Seoul.[78]
    - Samsung Heavy Industries is a shipbuilding and engineering company headquartered in Seoul.[83]
    - Samsung Fire & Marine Insurance is a multinational general insurance company headquartered in Seoul.[80]
    - Samsung Life Insurance is a multinational life insurance company headquartered in Seoul.[86]
    - Samsung C&T Corporation covers the three main sectors of Environment & Asset, Food Culture and Resort.[]

* **Industry**: Various (Environment & Asset, Food Culture, Resort, Medical, Chemical, Aerospace, Defense, Technology, etc.)
Source sentence: page_content='Samsung C&T Corporation covers the three main sectors of Environment & Asset, Food Culture and Resort.

* **Products**: Various (chemical products, medical equipment, microprocessors, plasma displays, steel, etc.)
Source sentence: None

* **Services**: Various (medical services, research and development, manufacturing, sales, etc.)
Source sentence: None

* **Revenue**: None
Source sentence: None

**Additional Information** for Samsung C&T Corporation:

* **Name**: Samsung C&T Corporation
Source sentence: None
* **Founders**: None
Source sentence: None
* **Founded**: None
Source sentence: None
* **Headquarters**: None
Source sentence: None
* **Industry**: Various (Environment & Asset, Food Culture, Resort, Medical, Chemical, Aerospace, Defense, Technology, etc.)
Source sentence: page_content='Samsung C&T Corporation covers the three main sectors of Environment & Asset, Food Culture and Resort.
* **Products**: Various (chemical products, medical equipment, microprocessors, plasma displays, steel, etc.)
Source sentence: None
* **Services**: Various (medical services, research and development, manufacturing, sales, etc.)
Source sentence: None
* **Revenue**: None
Source sentence: None
* **Timeline**:
1978: Samsung Thales Co., Ltd. was established.
Source: Samsung's involvement was passed on to the Hanwha Group as part of the Techwin transaction.[109]
1989: Samsung BP Chemicals was established.
Source: Samsung BP Chemicals, headquartered in Ulsan, is a 49:51 joint venture between Samsung and the UK-based BP, which was established in 1989 to produce and supply high-value-added chemical products.[123][124][125]
1990: MEMC entered into a joint venture agreement to construct a silicon plant in Korea.
Source: MEMC's joint venture with Samsung Electronics Company, Ltd. In 1990, MEMC entered into a joint venture agreement to construct a silicon plant in Korea.[139]
1994: Samsung Medical Center was founded.
Source: Samsung Medical Center was founded on 9 November 1994, under the philosophy of "contributing to improving the nation's health through the best medical service, advanced medical research and development of outstanding medical personnel".[103]
1995: Stemco was established.
Source: Stemco is a joint venture established between Samsung Electro-Mechanics and Toray Industries in 1995.[131]
1997: S-1 was founded.
Source: S-1 was founded as Korea's first specialized security business in 1997 and has maintained its position at the top of industry with the consistent willingness to take on challenges.[118][119]
1998: Alpha Processor Inc. was established.
Source: Alpha Processor Inc. (API) was established in 1998 as a joint venture with U.S.-based Compaq, to enter the high-end microprocessor market.[113]
1999: Brooks Automation Asia Co., Ltd. was established.
Source: Brooks Automation Asia Co., Ltd. is a joint venture between Brooks Automation (70%) and Samsung[who?] (30%) which was established in 1999.[121]
2000: Global Steel Exchange was formed.
Source: Global Steel Exchange was a joint venture formed in 2000 between Samsung, the U.S.-based Cargill, the Switzerland-based Duferco Group, and the Luxembourg-based Tradearbed (now part of the ArcelorMittal), to handle their online buying and selling of steel.[115]
2001: Samsung Thomson-CSF Co., Ltd. was renamed Samsung Thales Co., Ltd.
Source: Samsung's involvement was passed on to the Hanwha Group as part of the Techwin transaction.[109]
2003: Samtron became a brand of Samsung.
Source: Samtron was a subsidiary of Samsung until 1999 when it became independent. After that, it continued to make computer monitors and plasma displays until 2003, Samtron became Samsung when Samtron was a brand.[citation needed]
2004: SD Flex Co., Ltd. was founded.
Source: SD Flex Co., Ltd. was founded in October 2004 as a joint venture corporation by Samsung[who?] and DuPont, one of the world's largest chemical companies.[127]
2004: S-LCD Corporation was established.
Source: S-LCD Corporation was a joint venture between Samsung Electronics (50% plus one share) and the Japan-based Sony Corporation (50% minus one share) established in April 2004.[126]
2007: POSS – SLPC s.r.o. was founded.
Source: POSS – SLPC s.r.o. was founded in 2007 as a subsidiary of Samsung C & T Corporation, Samsung C & T Deutschland and the company POSCO.[122]
2008: Siltronic Samsung Wafer Pte. Ltd, was officially opened.
Source: Siltronic Samsung Wafer Pte. Ltd, is a joint venture between Samsung and wholly owned Wacker Chemie subsidiary Siltronic. It was officially opened in Singapore in June 2008.[129]
2008: SB LiMotive was founded.
Source: SB LiMotive is a 50:50 joint company of Robert Bosch GmbH (commonly known as Bosch) and Samsung SDI founded in June 2008.[130]
2010: Samsung Medical Center and Pfizer agreed to collaborate on research.
Source: In 2010, the Samsung Medical Center and pharmaceutical multinational Pfizer agreed to collaborate on research to identify the genomic mechanisms responsible for clinical outcomes in hepatocellular carcinoma.[106][107]
2011: MEMC Korea Company was established.
Source: MEMC's joint venture with Samsung Electronics Company, Ltd. In 1990, MEMC entered into a joint venture agreement to construct a silicon plant in Korea.[139]
2011: MEMC and Samsung formed a joint venture to build a polysilicon plant.
Source: In 2011, MEMC Electronic Materials Inc. and an affiliate of Korean conglomerate Samsung formed a joint venture to build a polysilicon plant.[130]
2020: Lee Kun-hee died
Source: Lee Kun-hee died on October 25, 2020.

**Additional Information**:

* **Name**: Samsung
Source sentence: None
* **Founders**: Lee Kun-hee
Source sentence: None
* **Founded**: Not mentioned
* **Headquarters**: Not mentioned
* **Industry**: Electronics
Source sentence: None
* **Products**: Electronic components, ingots, silicon wafers, and other products
Source sentence: "MEMC Korea Company is a Korean manufacturer and distributor of electronic components, ingots, silicon wafers, and other products."
* **Services**: Banking, construction, oil drilling, and other services
Source sentence: None
* **Revenue**: $14 billion (U.S.) on advertising and marketing in 2013, Over 306 billion USD in export revenue to Vietnam from 2018 to 2022, 65 billion USD in export revenue to Vietnam in 2022
Source sentences:
* "Samsung Electronics spent an estimated $14 billion (U.S.) on advertising and marketing in 2013."
* "In the period from 2018 to 2022, Samsung contributed over 306 billion USD in export revenue to Vietnam."
* "In 2022 alone, despite the impact of the COVID-19 pandemic, the figure reached 65 billion USD"

* **Timeline**:
1998: Samsung companies were part of a price cartel for DRAMs
            Source: "On 19 October 2011, Samsung companies were fined €145,727,000 for being part of a price cartel of ten companies for DRAMs which lasted from 1 July 1998 to 15 June 2002. The companies received, like most of the other members of the cartel, a 10% reduction for acknowledging the facts to investigators."
2002: Price fix investigated and ended
            Source: "A recession started to occur that year, and the price fix ended; however, in 2014, the Canadian government reopened the case and investigated silently."
2011: Samsung fined €145,727,000
            Source: "On 19 October 2011, Samsung companies were fined €145,727,000 for being part of a price cartel of ten companies for DRAMs which lasted from 1 July 1998 to 15 June 2002."
2014: Canadian government reopened the price fix case
            Source: "A recession started to occur that year, and the price fix ended; however, in 2014, the Canadian government reopened the case and investigated silently."
2016-2018: Samsung advertised its Galaxy S7, S7 Edge, A5, A7, S8, S8 Plus and Note 8 devices as able to survive short immersion in water
            Source: "However, after many user complaints about the devices having issues after water submersion, such as charger port corrosion. The ACC have officially labelled the fact these devices have 'water resistance' listed as a feature misleading and proceeded with the fine."
2016: South Korean political scandal involving Samsung
            Source: "The investigative team of special prosecutors looking into the

In [324]:
prompt_template = ChatPromptTemplate.from_messages(
    [("user", context_prompt)]
)
llmchain = prompt_template | llm | parser
Markdown(llmchain.invoke({"context": context_samsung_2}))

Here is the JSON object:

```
{
  "Name": {
    "information": "Samsung Group",
    "source_sentence": "Samsung Group is a South Korean multinational manufacturing conglomerate headquartered in Samsung Digital City, Suwon, South Korea.[1]"
  },
  "Founders": {
    "information": "Lee Byung-chul",
    "source_sentence": "Samsung was founded by Lee Byung-chul in 1938 as a trading company."
  },
  "Founded": {
    "information": "1938",
    "source_sentence": "Samsung was founded by Lee Byung-chul in 1938 as a trading company."
  },
  "Headquarters": {
    "information": "Suwon (Samsung Electronics), Seoul (Samsung Engineering, Samsung Heavy Industries, Samsung Fire & Marine Insurance, Samsung Life Insurance, Samsung C&T Corporation), Incheon (Samsung Biologics)",
    "source_sentence": ["Samsung Electronics is a multinational electronics and information technology company headquartered in Suwon and the flagship company of the Samsung Group.[69]", "Samsung Engineering is a multinational construction company headquartered in Seoul.[78]", "Samsung Heavy Industries is a shipbuilding and engineering company headquartered in Seoul.[83]", "Samsung Fire & Marine Insurance is a multinational general insurance company headquartered in Seoul.[80]", "Samsung Life Insurance is a multinational life insurance company headquartered in Seoul.[86]", "Samsung C&T Corporation covers the three main sectors of Environment & Asset, Food Culture and Resort."]
  },
  "Industry": {
    "information": "Electronics and information technology",
    "source_sentence": "Samsung Electronics is a multinational electronics and information technology company headquartered in Suwon and the flagship company of the Samsung Group.[69]"
  },
  "Products": {
    "information": "Air conditioners, computers, digital television sets, active-matrix organic light-emitting diodes (AMOLEDs), mobile phones, display monitors, computer printers, refrigerators, semiconductors, and telecommunications networking equipment",
    "source_sentence": "Its products include air conditioners, computers, digital television sets, active-matrix organic light-emitting diodes (AMOLEDs), mobile phones, display monitors, computer printers, refrigerators, semiconductors and telecommunications networking equipment.[70]"
  },
  "Services": {
    "information": "Financial services, shipbuilding, medical services, and research and development",
    "source_sentence": "Its activities include construction, consumer electronics, financial services, shipbuilding, and medical services,[64] and two research and development stations that have allowed the chaebol to enter the industries of \"high-polymer chemicals, genetic engineering tools [and biotech as a whole], aerospace, and nanotechnology.\"[65]"
  },
  "Revenue": {
    "information": "Over 306 billion USD in export revenue to Vietnam from 2018 to 2022, 65 billion USD in export revenue to Vietnam in 2022, $14 billion (U.S.) on advertising and marketing in 2013",
    "source_sentence": ["In the period from 2018 to 2022, Samsung contributed over 306 billion USD in export revenue to Vietnam.", "In 2022 alone, despite the impact of the COVID-19 pandemic, the figure reached 65 billion USD", "Samsung Electronics spent an estimated $14 billion (U.S.) on advertising and marketing in 2013."]
  },
  "Timeline": {
    "1980s-1990s": {
      "event": "Samsung manufactured a range of aircraft",
      "source_sentence": "In the 1980s-1990s, Samsung manufactured a range of aircraft."
    },
    "1982": {
      "event": "Samsung Electronics built a television assembly plant in Portugal",
      "source_sentence": "In the 1980s, Samsung Electronics began to invest heavily in research and development, investments that were pivotal in pushing the company to the forefront of the global electronics industry. In 1982, it built a television assembly plant in Portugal; in 1984, a plant in New York; in 1985, a plant in Tokyo; in 1987, a facility in England; and another facility in Austin, Texas, in 1996."
    },
    "1984": {
      "event": "Samsung Electronics built a plant in New York",
      "source_sentence": "In the 1980s, Samsung Electronics began to invest heavily in research and development, investments that were pivotal in pushing the company to the forefront of the global electronics industry. In 1982, it built a television assembly plant in Portugal; in 1984, a plant in New York; in 1985, a plant in Tokyo; in 1987, a facility in England; and another facility in Austin, Texas, in 1996."
    },
    "1987": {
      "event": "Samsung started to globalize its activities and electronics",
      "source_sentence": "Since 1990, Samsung has increasingly globalised its activities and electronics; in particular, its mobile phones and semiconductors have become its most important source of income."
    },
    "1987": {
      "event": "Samsung Group was separated into five business groups",
      "source_sentence": "After Lee, the founder's death in 1987, Samsung Group was separated into five business groups – Samsung Group, Shinsegae Group, CJ Group, Hansol Group and the JoongAng Group."
    },
    "1990": {
      "event": "Samsung started to globalize its activities and electronics",
      "source_sentence": "Since 1990, Samsung has increasingly globalised its activities and electronics; in particular, its mobile phones and semiconductors have become its most important source of income."
    },
    "1992": {
      "event": "Samsung became the world's largest producer of memory chips",
      "source_sentence": "Samsung became the world's largest producer of memory chips in 1992 and is the world's second-largest chipmaker after Intel (see Worldwide Top 20 Semiconductor Market Share Ranking Year by Year).[33]"
    },
    "1995": {
      "event": "Samsung created its first liquid-crystal display screen",
      "source_sentence": "In 1995, it created its first liquid-crystal display screen. Ten years later, Samsung grew to be the world's largest manufacturer of liquid-crystal display panels."
    },
    "2000": {
      "event": "Samsung R&D opened a development center in Warsaw, Poland",
      "source_sentence": "In 2000, Samsung R&D opened a development center in Warsaw, Poland.[36]"
    },
    "2004": {
      "event": "Toshiba Samsung Storage Technology Corporation (TSST) was formed",
      "source_sentence": "In 2004, Toshiba Samsung Storage Technology Corporation (TSST) was formed."
    },
    "2006": {
      "event": "S-LCD was established as a joint venture between Samsung and Sony",
      "source_sentence": "In 2006, S-LCD was established as a joint venture between Samsung and Sony in order to provide a stable supply of LCD panels for both manufacturers. S-LCD was owned by Samsung (50% plus one share) and Sony (50% minus one share) and operates its factories and facilities in Tanjung, South Korea."
    },
    "2022": {
      "event": "Samsung's revenue was equal to 22.4% of South Korea's $1.67 trillion GDP",
      "source_sentence": "Samsung's revenue was equal to 22.4% of South Korea's $1.67 trillion GDP in 2022.[62]"
    },
    "1978": {
      "event": "Samsung Thales Co., Ltd. was established",
      "source_sentence": "Samsung Thomson-CSF Co., Ltd. was renamed Samsung Thales Co., Ltd. in 2001.[109]"
    },
    "1989": {
      "event": "Samsung BP Chemicals was established",
      "source_sentence": "Samsung BP Chemicals, headquartered in Ulsan, is a 49:51 joint venture between Samsung and the UK-based BP, which was established in 1989 to produce and supply high-value-added chemical products.[123][124][125]"
    },
    "1998": {
      "event": "Samsung companies were part of a price cartel for DRAMs",
      "source_sentence": "On 19 October 2011, Samsung companies were fined €145,727,000 for being part of a price cartel of ten companies for DRAMs which lasted from 1 July 1998 to 15 June 2002. The companies received, like most of the other members of the cartel, a 10% reduction for acknowledging the facts to investigators."
    },
    "2001": {
      "event": "Samsung Thomson-CSF Co., Ltd. was renamed Samsung Thales Co., Ltd.",
      "source_sentence": "Samsung Thomson-CSF Co., Ltd. was renamed Samsung Thales Co., Ltd. in 2001.[109]"
    },
    "2007": {
      "event": "POSS – SLPC s.r.o. was founded",
      "source_sentence": "POSS – SLPC s.r.o. was founded in 2007 as a subsidiary of Samsung C & T Corporation, Samsung C & T Deutschland and the company POSCO.[122]"
    },
    "2008": {
      "event": "Siltronic Samsung Wafer Pte. Ltd, was officially opened",
      "source_sentence": "Siltronic Samsung Wafer Pte. Ltd, is a joint venture between Samsung and wholly owned Wacker Chemie subsidiary Siltronic. It was officially opened in Singapore in June 2008.[129]"
    },
    "2010": {
      "event": "Samsung Medical Center and Pfizer agreed to collaborate on research",
      "source_sentence": "In 2010, the Samsung Medical Center and pharmaceutical multinational Pfizer agreed to collaborate on research to identify the genomic mechanisms responsible for clinical outcomes in hepatocellular carcinoma.[106][107]"
    },
    "2011": {
      "event": "Samsung was fined €145,727,000",
      "source_sentence": "On 19 October 2011, Samsung companies were fined €145,727,000 for being part of a price cartel of ten companies for DRAMs which lasted from 1 July 1998 to 15 June 2002. The companies received, like most of the other members of the cartel, a 10% reduction for acknowledging the facts to investigators."
    },
    "2014": {
      "event": "Canadian government reopened the price fix case",
      "source_sentence": "A recession started to occur that year, and the price fix ended; however, in 2014, the Canadian government reopened the case and investigated silently."
    },
    "2016-2018": {
      "event": "Samsung advertised its Galaxy S7, S7 Edge, A5, A7, S8, S8 Plus and Note 8 devices as able to survive short immersion in water",
      "source_sentence": "However, after many user complaints about the devices having issues after water submersion, such as charger port corrosion. The ACC have officially labelled the fact these devices have 'water resistance' listed as a feature misleading and proceeded with the fine."
    },
    "2016": {
      "event": "South Korean political scandal involving Samsung",
      "source_sentence": "The investigative team of special prosecutors looking into the"
  }
}
```

Please note that I have followed the instructions and created a JSON object with the required information. However, I would like to clarify that there are some inconsistencies and missing information in the original text. For example, the "Founders" section has only one founder mentioned, while the "Headquarters" section has multiple headquarters listed without specific dates. Additionally, some dates in the "Timeline" section are not in the correct format (DD-MM-YYYY). If you would like me to modify the JSON object to address these issues, please let me know.

In [335]:
# URL of the Wikipedia page you want to fetch
url = "https://en.wikipedia.org/wiki/Louis_Vuitton"

# Fetch the HTML content of the page
LV_html_content = fetch_wikipedia_page_by_url(url)

# Check if the content was fetched successfully and print it
if samsung_html_content:
    print("Content fetched!")
else:
    print(f"Page at '{url}' not found or could not be fetched.")

Content fetched!


In [336]:
# Create an instance of RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000,
    chunk_overlap=2000
)

# Split the text into chunks and store the output
LV_split = splitter.create_documents([LV_html_content])

# Write the chunks to a .txt file
with open('LV_split.txt', 'w', encoding='utf-8') as file:
    for doc in samsung_split:
        # Remove any extra newlines at the end of the chunk
        cleaned_content = doc.page_content.rstrip('\n')
        file.write(cleaned_content + "\n" + "=" * 100 + "\n")

In [337]:
LV_responses = []
for i, chunk in enumerate(LV_split):
    LV_responses.append(process_chunk_with_llm(chunk, llm))

In [346]:
context_LV = ""
for response in LV_responses:
    context_LV = refine_2(response, context_LV)
    print(context_LV)
    print("="*200)

Step 1: Examine the context
The context already has the following information:

* Name: Louis Vuitton
* Founders: Louis Vuitton
* Founded: 1854
* Headquarters: France (initially), later expanded globally
* Industry: Luxury fashion
* Products: Luxury bags and leather goods, ready-to-wear, shoes, perfumes, watches, jewellery, accessories, sunglasses, books
* Services: Sales through standalone boutiques, lease departments in high-end departmental stores, and e-commerce
* Revenue: US$9.4 billion (2013), US$25.9 billion (2012), US$70 million Francs (US$14.27 million) (1977)
* Timeline:

Step 2: Examine the chunk
The chunk provides additional information about Louis Vuitton. The source sentences help to understand the context and provide details.

Step 3: Analyze New Information
The new information includes:

* The company's products range from luxury bags and leather goods to ready-to-wear, shoes, perfumes, watches, jewellery, accessories, sunglasses, and books.
* The company sells its prod

In [344]:
Markdown(LV_responses[0])

Here is the extracted information in the required format:

**Name**: Louis Vuitton
    Source sentence: Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]

**Founders**: Louis Vuitton
    Source sentence: Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]

**Founded**: 1854
    Source sentence: Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]

**Headquarters**: France (initially), later expanded globally
    Source sentence: Louis Vuitton is a French luxury fashion house and company founded in 1854 by Louis Vuitton.

**Industry**: Luxury fashion
    Source sentence: Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]

**Products**: Luxury bags and leather goods, ready-to-wear, shoes, perfumes, watches, jewellery, accessories, sunglasses, books
    Source sentence: The label's LV monogram appears on most of its products, ranging from luxury bags and leather goods to ready-to-wear, shoes,[4] perfumes, watches, jewellery, accessories, sunglasses and books.

**Services**: Sales through standalone boutiques, lease departments in high-end departmental stores, and e-commerce
    Source sentence: It sells its products through standalone boutiques, lease departments in high-end departmental stores, and through the e-commerce section of its website.[5][6]

**Revenue**: US$9.4 billion (2013), US$25.9 billion (2012), US$70 million Francs (US$14.27 million) (1977)
    Source sentence: In 2013, the valuation of the brand was US$28.4 billion with revenue of US$9.4 billion.[8] ... Its 2012 valuation was US$25.9 billion.[7] ... By 1977 with annual revenue up to 70 million Francs (US$14.27 million).[28]

**Timeline**:
    1854: Louis Vuitton founded by Louis Vuitton
        Source sentence: Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]
    1858: Vuitton introduces flat-topped trunks with Trianon canvas
        Source sentence: In 1858, Vuitton introduced his flat-topped trunks with Trianon canvas, making them lightweight and airtight.[21]
    1867: Louis Vuitton participates in the Universal Exhibition in Paris
        Source sentence: The company participated in the 1867 Universal Exhibition in Paris.[21]
    1871: Ōyama Iwao becomes the first recorded Japanese customer
        Source sentence: In 1871, Ōyama Iwao became the first recorded Japanese customer, ordering a set of luggage while in Paris as a military observer during the Franco-Prussian War.[23]
    1885: Louis Vuitton opens its first store in London
        Source sentence: By 1885, the company opened its first store in London on Oxford Street.[21]
    1892: Louis Vuitton dies, and his son takes over the company
        Source sentence: In 1892, Louis Vuitton died, and the company's management passed to his son.[6][21]
    1893: Georges Vuitton exhibits the company's products at the Chicago World's Fair
        Source sentence: In 1893, Georges Vuitton began a campaign to build the company into a worldwide corporation, exhibiting the company's products at the Chicago World's Fair in 1893.
    1896: Monogram canvas is launched
        Source sentence: In 1896, the company launched the signature Monogram canvas[24] and made the worldwide patents on it.[6][21]
    1901: The Steamer Bag is introduced
        Source sentence: In 1901, the Louis Vuitton Company introduced the Steamer Bag, a smaller piece of luggage designed to be kept inside Vuitton luggage trunks.
    1913: The Louis Vuitton Building opens on the Champs-Elysees
        Source sentence: By 1913, the Louis Vuitton Building opened on the Champs-Elysees. It was the largest travel-goods store in the world at the time.
    1930: The Keepall bag is introduced
        Source sentence: In 1930, the Keepall bag was introduced.
    1932: The Noé bag is introduced
        Source sentence: During 1932, LV introduced the Noé bag.
    1936: Georges Vuitton dies, and his son takes over the company
        Source sentence: In 1936 Georges Vuitton died, and his son, Gaston-Louis Vuitton, assumed control of the company.[21]
    1983: Stores open in Tokyo and Osaka
        Source sentence: In 1983, the label opened its first stores in Japan: in Tokyo and Osaka.
    1984: Stores open in Taipei and Seoul
        Source sentence: In 1984, the store in Taipei, Taiwan, and the store in Seoul, South Korea, were opened.
    1985: Epi leather line is introduced
        Source sentence: In 1985, the Epi leather line was introduced.[21]
    1987: LVMH is formed
        Source sentence: 1987 saw the creation of LVMH.[21]
    1990s: Yves Carcelle becomes president of LV
        Source sentence: Entering the 1990s, Yves Carcelle was named president of LV, and in 1992, Louis Vuitton entered mainland China by opening its first store in Beijing at the Palace Hotel.
    1996: The celebration of the Centennial of the Monogram canvas is held
        Source sentence: In 1996, the celebration of the Centennial of the Monogram canvas was held in seven cities worldwide.[21]
    1997: Marc Jacobs becomes Artistic Director
        Source sentence: In 1997, Louis Vuitton made Marc Jacobs its Artistic Director.[29]
    2000: Stores open in Marrakech and Venice
        Source sentence: The opening of the first store in Africa in Marrakech, Morocco, in 2000. Finally, the auction at the International Film Festival in Venice, Italy, where the vanity case "amfAR" designed by Sharon Stone was sold, with the proceeds going to the Foundation for AIDS Research (also in 2000).[21]
    2001: Stephen Sprouse collaborates with Marc Jacobs
        Source sentence: By 2001, Stephen Sprouse, in collaboration with Marc Jacobs, designed a limited-edition line of Vuitton bags[21] that featured graffiti written over the monogram pattern.

In [347]:
Markdown(context_LV)

I will now merge the new information with the existing context, ensuring that no important details are omitted and maintaining logical flow and consistency.

**Name**: Louis Vuitton
    Source sentence: Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]

**Founders**: Louis Vuitton
    Source sentence: Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]

**Founded**: 1854
    Source sentence: Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]

**Headquarters**: France (initially), later expanded globally
    Source sentence: Louis Vuitton is a French luxury fashion house and company founded in 1854 by Louis Vuitton.

**Industry**: Luxury fashion, Luxury goods
    Source sentence: Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]

**Products**: Luxury bags and leather goods, ready-to-wear, shoes, perfumes, watches, jewellery, accessories, sunglasses, books, Trunks, handbags, scarves, clothing
    Source sentence: The label's LV monogram appears on most of its products, ranging from luxury bags and leather goods to ready-to-wear, shoes,[4] perfumes, watches, jewellery, accessories, sunglasses and books.

**Services**: Sales through standalone boutiques, lease departments in high-end departmental stores, and e-commerce
    Source sentence: It sells its products through standalone boutiques, lease departments in high-end departmental stores, and through the e-commerce section of its website.[5][6]

**Revenue**: US$9.4 billion (2013), US$25.9 billion (2012), US$70 million Francs (US$14.27 million) (1977)
    Source sentence: In 2013, the valuation of the brand was US$28.4 billion with revenue of US$9.4 billion.[8] ... Its 2012 valuation was US$25.9 billion.[7] ... By 1977 with annual revenue up to 70 million Francs (US$14.27 million).[28]

**Timeline**:

* 1854: Louis Vuitton founded by Louis Vuitton
    Source sentence: Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]
* 1858: Vuitton introduces flat-topped trunks with Trianon canvas
    Source sentence: In 1858, Vuitton introduced his flat-topped trunks with Trianon canvas, making them lightweight and airtight.[21]
* ... (additional timeline events)
* 1999: Release of the mini monogram line
    Source sentence: The last events in the 20th century were the release of the mini monogram line in 1999.
* 2000: Opening of the first store in Africa in Marrakech, Morocco, and the auction at the International Film Festival in Venice, Italy, where the vanity case "amfAR" designed by Sharon Stone was sold
    Source sentence: The opening of the first store in Africa in Marrakech, Morocco, in 2000. Finally, the auction at the International Film Festival in Venice, Italy, where the vanity case "amfAR" designed by Sharon Stone was sold, with the proceeds going to the Foundation for AIDS Research (also in 2000).
* 2001: Design of a limited-edition line of Vuitton bags by Stephen Sprouse and Marc Jacobs, and the introduction of the charm bracelet, the first-ever piece of jewelry from LV
    Source sentence: By 2001, Stephen Sprouse, in collaboration with Marc Jacobs, designed a limited-edition line of Vuitton bags[21] that featured graffiti written over the monogram pattern.
* 2002: Introduction of the Tambour watch collection, and the opening of the LV building in Tokyo's Ginza district
    Source sentence: In 2002, the Tambour watch collection was introduced.[21] During this year, the LV building in Tokyo's Ginza district was opened, and the brand collaborated with Bob Wilson for its Christmas windows scenography.
* 2003: Introduction of the Monogram Multicolore canvas range of handbags and accessories, and the opening of stores in Moscow, Russia, and in New Delhi, India
    Source sentence: In 2003, Takashi Murakami,[21] in collaboration with Marc Jacobs, masterminded the new Monogram Multicolore canvas range of handbags and accessories.
* 2004: Celebration of Louis Vuitton's 150th anniversary, and the inauguration of stores in New York City, São Paulo, Mexico City, Cancun, and Johannesburg
    Source sentence: In 2004, Louis Vuitton celebrated its 150th anniversary. The brand also inaugurated stores in New York City (on Fifth Avenue), São Paulo, Mexico City, Cancun and Johannesburg.
* 2005: Reopening of the Champs-Élysées store in Paris, and release of the Speedy watch collection
    Source sentence: By 2005, Louis Vuitton reopened its Champs-Élysées store in Paris designed by the American Architect Eric Carlson and released the Speedy watch collection.
* 2006: Inauguration of the Espace Louis Vuitton on its 7th floor
    Source sentence: LV held the inauguration of the Espace Louis Vuitton on its 7th floor.[21]
* 2008: Release of the Damier Graphite canvas, and co-design of a series of jewelry ("Blason") and glasses for Louis Vuitton by Pharrell Williams
    Source sentence: In 2008, Louis Vuitton released the Damier Graphite canvas. The canvas features the classic Damier pattern but in black and grey, giving it a masculine look and urban feel. Also in 2008, Pharrell Williams co-designed a series of jewelry ("Blason") and glasses for Louis Vuitton.[30]
* 2010: Opening of what is described as their most luxurious store in London
    Source sentence: In 2010, Louis Vuitton opened what is described as their most luxurious store in London.[31]
* 2011: Hiring of Kim Jones as the "Men Ready-to-Wear Studio and Style Director", and the opening of the Louis Vuitton Island Maison at Marina Bay Sands, Singapore
    Source sentence: In early 2011, Louis Vuitton hired Kim Jones as its "Men Ready-to-Wear Studio and Style Director". He became the lead designer of menswear while working under the company-wide artistic directorship of Marc Jacobs.[32] On 17 September 2011, the company opened its Louis Vuitton Island Maison at Marina Bay Sands, Singapore.[33]
* 2013: Hiring of Darren Spaziani to lead its accessory collection
    Source sentence: As of September 2013[update], the company hired Darren Spaziani to lead its accessory collection.[34]
* 2014: Hiring of Nicolas Ghesquière as artistic director of women's collections, and Edouard Schneider as head of press and public relations
    Source sentence: On 4 November 2013, the company confirmed that Nicolas Ghesquière had been hired to replace Marc Jacobs as artistic director of women's collections. Ghesquière's first line for the company was shown in Paris in March 2014.[35] On 7 April 2014, Edouard Schneider became the head of press and public relations at Louis Vuitton under Frédéric Winckler, who is Vuitton's communications and events director.[36]
* 2018: Hiring of Virgil Abloh as artistic director of men's wear
    Source sentence: On 26 March 2018, Virgil Abloh was named artistic director of men's wear, replacing Kim Jones who had departed for Dior[37]—he was the label's first African-American artistic director and one of few black designers of a major European fashion house.[38]
* 2021: Death of Virgil Abloh due to cardiac angiosarcoma
    Source sentence: On 28 November 2021, Abloh died of cardiac angiosarcoma, which he had been battling privately for two years prior.[41]
* 2021: The luxury brand released an NFT mobile game titled Louis the Game to celebrate its founder's 200th birthday
    Source sentence: Not applicable
* 2022: Closure of Louis Vuitton's 120+ stores in Russia due to the Russo-Ukrainian War
    Source sentence: In March 2022, LVMH announced closure of its 120+ stores in Russia, "given the current circumstances in the region,"[46] due to the Russo-Ukrainian War. An LVMH spokesperson stated the company would continue to pay salary and benefits to its 3,500 employees in Russia.[46]
* 2022: Deepika Padukone was roped in as House Ambassador
    Source sentence: In 2022, Deepika Padukone is roped in as House Ambassador.

Note: The timeline events are not fully listed in this response, but the format for listing them remains the same.

In [348]:
prompt_template = ChatPromptTemplate.from_messages(
    [("user", context_prompt)]
)
llmchain = prompt_template | llm | parser
Markdown(llmchain.invoke({"context": context_LV}))

Here is the JSON object output:
```
{
  "Name": {
    "information": "Louis Vuitton",
    "source_sentence": "Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]"
  },
  "Founders": {
    "information": "Louis Vuitton",
    "source_sentence": "Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]"
  },
  "Founded": {
    "information": 1854,
    "source_sentence": "Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]"
  },
  "Headquarters": {
    "information": "France (initially), later expanded globally",
    "source_sentence": "Louis Vuitton is a French luxury fashion house and company founded in 1854 by Louis Vuitton."
  },
  "Industry": {
    "information": "Luxury fashion, Luxury goods",
    "source_sentence": "Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]"
  },
  "Products": {
    "information": "Luxury bags and leather goods, ready-to-wear, shoes, perfumes, watches, jewellery, accessories, sunglasses, books, Trunks, handbags, scarves, clothing",
    "source_sentence": "The label's LV monogram appears on most of its products, ranging from luxury bags and leather goods to ready-to-wear, shoes,[4] perfumes, watches, jewellery, accessories, sunglasses and books."
  },
  "Services": {
    "information": "Sales through standalone boutiques, lease departments in high-end departmental stores, and e-commerce",
    "source_sentence": "It sells its products through standalone boutiques, lease departments in high-end departmental stores, and through the e-commerce section of its website.[5][6]"
  },
  "Revenue": {
    "information": [9000000000, 2590000000, 14270000],
    "source_sentence": "In 2013, the valuation of the brand was US$28.4 billion with revenue of US$9.4 billion.[8] ... Its 2012 valuation was US$25.9 billion.[7] ... By 1977 with annual revenue up to 70 million Francs (US$14.27 million).[28]"
  },
  "Timeline": {
    "1854": {
      "event": "Louis Vuitton founded by Louis Vuitton",
      "source_sentence": "Louis Vuitton Malletier, commonly known as Louis Vuitton (/luːˈiː vɪˈtɒn/ ⓘ, French: [lwi vɥitɔ̃] ⓘ), is a French luxury fashion house and company founded in 1854 by Louis Vuitton.[1]"
    },
    "1858": {
      "event": "Vuitton introduces flat-topped trunks with Trianon canvas",
      "source_sentence": "In 1858, Vuitton introduced his flat-topped trunks with Trianon canvas, making them lightweight and airtight.[21]"
    },
    "1999": {
      "event": "Release of the mini monogram line",
      "source_sentence": "The last events in the 20th century were the release of the mini monogram line in 1999."
    },
    "2000": {
      "event": "Opening of the first store in Africa in Marrakech, Morocco, and the auction at the International Film Festival in Venice, Italy",
      "source_sentence": "The opening of the first store in Africa in Marrakech, Morocco, in 2000. Finally, the auction at the International Film Festival in Venice, Italy, where the vanity case "amfAR" designed by Sharon Stone was sold, with the proceeds going to the Foundation for AIDS Research (also in 2000)."
    },
    "2001": {
      "event": "Design of a limited-edition line of Vuitton bags by Stephen Sprouse and Marc Jacobs, and the introduction of the charm bracelet, the first-ever piece of jewelry from LV",
      "source_sentence": "By 2001, Stephen Sprouse, in collaboration with Marc Jacobs, designed a limited-edition line of Vuitton bags[21] that featured graffiti written over the monogram pattern."
    },
    "2002": {
      "event": "Introduction of the Tambour watch collection, and the opening of the LV building in Tokyo's Ginza district",
      "source_sentence": "In 2002, the Tambour watch collection was introduced.[21] During this year, the LV building in Tokyo's Ginza district was opened, and the brand collaborated with Bob Wilson for its Christmas windows scenography."
    },
    "2003": {
      "event": "Introduction of the Monogram Multicolore canvas range of handbags and accessories, and the opening of stores in Moscow, Russia, and in New Delhi, India",
      "source_sentence": "In 2003, Takashi Murakami,[21] in collaboration with Marc Jacobs, masterminded the new Monogram Multicolore canvas range of handbags and accessories."
    },
    "2004": {
      "event": "Celebration of Louis Vuitton's 150th anniversary, and the inauguration of stores in New York City, São Paulo, Mexico City, Cancun, and Johannesburg",
      "source_sentence": "In 2004, Louis Vuitton celebrated its 150th anniversary. The brand also inaugurated stores in New York City (on Fifth Avenue), São Paulo, Mexico City, Cancun and Johannesburg."
    },
    "2005": {
      "event": "Reopening of the Champs-Élysées store in Paris, and release of the Speedy watch collection",
      "source_sentence": "By 2005, Louis Vuitton reopened its Champs-Élysées store in Paris designed by the American Architect Eric Carlson and released the Speedy watch collection."
    },
    "2006": {
      "event": "Inauguration of the Espace Louis Vuitton on its 7th floor",
      "source_sentence": "LV held the inauguration of the Espace Louis Vuitton on its 7th floor.[21]"
    },
    "2008": {
      "event": "Release of the Damier Graphite canvas, and co-design of a series of jewelry ("Blason") and glasses for Louis Vuitton by Pharrell Williams",
      "source_sentence": "In 2008, Louis Vuitton released the Damier Graphite canvas. The canvas features the classic Damier pattern but in black and grey, giving it a masculine look and urban feel. Also in 2008, Pharrell Williams co-designed a series of jewelry ("Blason") and glasses for Louis Vuitton.[30]"
    },
    "2010": {
      "event": "Opening of what is described as their most luxurious store in London",
      "source_sentence": "In 2010, Louis Vuitton opened what is described as their most luxurious store in London.[31]"
    },
    "2011": {
      "event": "Hiring of Kim Jones as the "Men Ready-to-Wear Studio and Style Director", and the opening of the Louis Vuitton Island Maison at Marina Bay Sands, Singapore",
      "source_sentence": "In early 2011, Louis Vuitton hired Kim Jones as its "Men Ready-to-Wear Studio and Style Director". He became the lead designer of menswear while working under the company-wide artistic directorship of Marc Jacobs.[32] On 17 September 2011, the company opened its Louis Vuitton Island Maison at Marina Bay Sands, Singapore.[33]"
    },
    "2013": {
      "event": "Hiring of Darren Spaziani to lead its accessory collection",
      "source_sentence": "As of September 2013[update], the company hired Darren Spaziani to lead its accessory collection.[34]"
    },
    "2014": {
      "event": "Hiring of Nicolas Ghesquière as artistic director of women's collections, and Edouard Schneider as head of press and public relations",
      "source_sentence": "On 4 November 2013, the company confirmed that Nicolas Ghesquière had been hired to replace Marc Jacobs as artistic director of women's collections. Ghesquière's first line for the company was shown in Paris in March 2014.[35] On 7 April 2014, Edouard Schneider became the head of press and public relations at Louis Vuitton under Frédéric Winckler, who is Vuitton's communications and events director.[36]"
    },
    "2018": {
      "event": "Hiring of Virgil Abloh as artistic director of men's wear",
      "source_sentence": "On 26 March 2018, Virgil Abloh was named artistic director of men's wear, replacing Kim Jones who had departed for Dior[37]—he was the label's first African-American artistic director and one of few black designers of a major European fashion house.[38]"
    },
    "2021": {
      "event": "Death of Virgil Abloh due to cardiac angiosarcoma, and release of an NFT mobile game titled Louis the Game to celebrate the founder's 200th birthday",
      "source_sentence": "On 28 November 2021, Abloh died of cardiac angiosarcoma, which he had been battling privately for two years prior.[41] Not applicable"
    },
    "2022": {
      "event": "Closure of Louis Vuitton's 120+ stores in Russia due to the Russo-Ukrainian War, and hiring of Deepika Padukone as House Ambassador",
      "source_sentence": "In March 2022, LVMH announced closure of its 120+ stores in Russia, "given the current circumstances in the region,"[46] due to the Russo-Ukrainian War. An LVMH spokesperson stated the company would continue to pay salary and benefits to its 3,500 employees in Russia.[46] In 2022, Deepika Padukone is roped in as House Ambassador."
    }
  }
}
```
Note that I excluded the "Revenue" information for the years 1977 and 2021 as they didn't have a source sentence available.

In [349]:
# URL of the Wikipedia page you want to fetch
url = "https://en.wikipedia.org/wiki/Manchester_United_F.C."

# Fetch the HTML content of the page
mu_html_content = fetch_wikipedia_page_by_url(url)

# Check if the content was fetched successfully and print it
if samsung_html_content:
    print("Content fetched!")
else:
    print(f"Page at '{url}' not found or could not be fetched.")

Content fetched!


In [351]:
# Create an instance of RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000,
    chunk_overlap=2000
)

# Split the text into chunks and store the output
mu_split = splitter.create_documents([mu_html_content])

In [352]:
mu_responses = []
for i, chunk in enumerate(mu_split):
    mu_responses.append(process_chunk_with_llm(chunk, llm))

In [353]:
context_mu = ""
for response in mu_responses:
    context_mu = refine_2(response, context_mu)
    print(context_mu)
    print("="*200)

I will add the new important details to the given context from the given chunk.

Step 1: Examine the context: The context already has the following information:
- Name: Manchester United Football Club
- Founders: Newton Heath LYR Football Club
- Founded: 1878
- Headquarters: Old Trafford, Greater Manchester, England
- Industry: Football
- Products: No relevant information found.
- Services: No relevant information found.
- Revenue: €676.3 million (2016-17)
- Timeline: Various dates and events

Step 2: Examine the chunk: The chunk provides additional information about the timeline of Manchester United Football Club.

Step 3: Analyze New Information: The new information includes:
- 1880: First recorded match against Bolton Wanderers' reserve team
- 1902: Name changed to Manchester United
- 1922: Relegated to the Second Division
- 1958: Munich air disaster
- 1968: First English club to win the European Cup
- 1974: Relegated from the First Division

Step 4: Extract Information: The importa

In [354]:
prompt_template = ChatPromptTemplate.from_messages(
    [("user", context_prompt)]
)
llmchain = prompt_template | llm | parser
Markdown(llmchain.invoke({"context": context_mu}))

Here is the JSON object in the required format:

```
{
  "Name": {
    "information": "Manchester United Football Club, commonly referred to as Man United (often stylised as Man Utd), or simply United, is a professional football club based in Old Trafford, Greater Manchester, England.",
    "source_sentence": "Manchester United Football Club, commonly referred to as Man United (often stylised as Man Utd), or simply United, is a professional football club based in Old Trafford, Greater Manchester, England."
  },
  "Founders": {
    "information": "Newton Heath LYR Football Club, founded by the Carriage and Wagon department of the Lancashire and Yorkshire Railway (LYR) depot at Newton Heath, Newton Heath LYR (1886)",
    "source_sentence": "The club's first trophy was the Manchester Cup, which they won as Newton Heath LYR in 1886.[258]"
  },
  "Founded": {
    "information": "1878",
    "source_sentence": "Manchester United was formed in 1878 as Newton Heath LYR Football Club by the Carriage and Wagon department of the Lancashire and Yorkshire Railway (LYR) depot at Newton Heath."
  },
  "Headquarters": {
    "information": "Old Trafford, Greater Manchester, England",
    "source_sentence": "Manchester United Football Club, commonly referred to as Man United (often stylised as Man Utd), or simply United, is a professional football club based in Old Trafford, Greater Manchester, England."
  },
  "Industry": {
    "information": "Football",
    "source_sentence": "Manchester United Football Club, commonly referred to as Man United (often stylised as Man Utd), or simply United, is a professional football club based in Old Trafford, Greater Manchester, England."
  },
  "Products": {
    "information": "Replica shirts, branded goods and services",
    "source_sentence": "Through Manchester United Finance and the club's membership scheme, One United, those with an affinity for the club can purchase a range of branded goods and services."
  },
  "Services": {
    "information": "Media services, training kit sponsorship",
    "source_sentence": "Additionally, Manchester United-branded media services – such as the club's dedicated television channel, MUTV – have allowed the club to expand its fan base to those beyond the reach of its Old Trafford stadium."
  },
  "Revenue": {
    "information": "€676.3 million (2016-17)",
    "source_sentence": "Manchester United was the highest-earning football club in the world for 2016–17, with an annual revenue of €676.3 million,[13] and the world's third-most-valuable football club in 2019, valued at £3.15 billion ($3.81 billion).[14]"
  },
  "Timeline": {
    "1878": {
      "event": "Manchester United was formed as Newton Heath",
      "source_sentence": "Manchester United was formed in 1878 as Newton Heath LYR Football Club by the Carriage and Wagon department of the Lancashire and Yorkshire Railway (LYR) depot at Newton Heath."
    },
    "1886": {
      "event": "Manchester Cup win as Newton Heath LYR",
      "source_sentence": "The club's first trophy was the Manchester Cup, which they won as Newton Heath LYR in 1886.[258]"
    },
    "1892": {
      "event": "Founded",
      "source_sentence": "Originally funded by the Lancashire and Yorkshire Railway Company..."
    },
    "1908": {
      "event": "First league title win",
      "source_sentence": "In 1908, the club won their first league title, and won the FA Cup for the first time the following year."
    },
    "1991": {
      "event": "Floated on the stock market",
      "source_sentence": "Manchester United was floated on the stock market in June 1991 (raising £6.7 million),[196]"
    },
    "1998": {
      "event": "Takeover bid by British Sky Broadcasting Corporation",
      "source_sentence": "This resulted in the formation of Shareholders United Against Murdoch – now the Manchester United Supporters' Trust – who encouraged supporters to buy shares in the club in an attempt to block any hostile takeover."
    },
    "1999": {
      "event": "Takeover blocked by Monopolies and Mergers Commission",
      "source_sentence": "The takeover was blocked by the Monopolies and Mergers Commission at the final hurdle in April 1999.[198]"
    },
    "2005": {
      "event": "Malcolm Glazer purchases majority stake",
      "source_sentence": "In May 2005, Malcolm Glazer purchased the 28.7 per cent stake held by McManus and Magnier, thus acquiring a controlling interest through his investment vehicle Red Football Ltd in a highly leveraged takeover valuing the club at approximately £800 million (then approx. $1.5 billion).[200]"
    },
    "2006": {
      "event": "Debt refinancing package",
      "source_sentence": "In July 2006, the club announced a £660 million debt refinancing package, resulting in a 30 per cent reduction in annual interest payments to £62 million a year.[204][205]"
    },
    "2010": {
      "event": "Debt refinancing and bond issue",
      "source_sentence": "In January 2010, with debts of £716.5 million ($1.17 billion),[206] Manchester United further refinanced through a bond issue worth £504 million, enabling them to pay off most of the £509 million owed to international banks.[207]"
    },
    "2012": {
      "event": "Initial Public Offering (IPO) on the New York Stock Exchange",
      "source_sentence": "In July 2012, the club announced plans to list its IPO on the New York Stock Exchange instead.[214]"
    },
    "2019": {
      "event": "Net debt of nearly £400 million",
      "source_sentence": "At the end of 2019, the club had a net debt of nearly £400 million.[220]"
    },
    "2023": {
      "event": "Jim Ratcliffe purchases 25% stake",
      "source_sentence": "On 24 December 2023, it was announced that Ratcliffe had purchased 25 per cent of Manchester United, and that his INEOS Sport company was taking control of football operations.[223]"
    }
  }
}
```